In [ ]:
using PyCall
using PyPlot

using Statistics
using DelimitedFiles
using Distributions

using OrderedCollections

using Dates

using Plots, Measures

yf = pyimport("yfinance")
np = pyimport("numpy")
pd = pyimport("pandas")
pm = pyimport("pymarkowitz")

include("OptimPortfolio.jl")
include("Utils.jl")

rcParams = PyDict(matplotlib["rcParams"])

rcParams["font.size"] = 20
rcParams["axes.labelsize"] = 30

rcParams["xtick.labelsize"] = 16
rcParams["ytick.labelsize"] = 16

rcParams["figure.figsize"] = (8,6)

L2D = PyPlot.matplotlib.lines.Line2D
Ellipse = PyPlot.matplotlib.patches.Ellipse

# Data download and basic analysis 

In [ ]:
start = "2013-01-01"
finish = "2017-01-01" 

start_Fortune = "1990-01-01"
finish_Fortune = "2022-01-01"

start_test = "2017-01-01"
finish_test_1  ="2018-01-01"
finish_test_2  ="2019-01-01"
finish_test_3  ="2020-01-01"

assets = ["AAPL", "MSFT", "AMZN",  "TSLA", "GOOGL",  "GOOG", "UNH", "NVDA", "JNJ", "META"] #"^GSPC"
assets_Fortune = ["WMT", "XOM", "F", "GE", "IBM", "C", "T", "MO", "BA", "BAC", "HPQ", "KR", "AIG", "HD", "PG"]

df = yf.download(assets, start, finish, progress=false)
df_Fortune = yf.download(assets_Fortune, start_Fortune, finish_Fortune, progress=false)

df_test_1 = yf.download(assets, start_test, finish_test_1, progress=false)
df_test_2 = yf.download(assets, start_test, finish_test_2, progress=false)
df_test_3 = yf.download(assets, start_test, finish_test_3, progress=false)

df_total = yf.download(assets, start, finish_test_3, progress=false)

market = yf.download("^GSPC", start, finish, progress=false)

market_test_1 = yf.download("^GSPC", start_test, finish_test_1, progress=false)
market_test_2 = yf.download("^GSPC", start_test, finish_test_2, progress=false)
market_test_3 = yf.download("^GSPC", start_test, finish_test_3, progress=false)

market_total = yf.download("^GSPC", start, finish_test_3, progress=false)

df_log_ret = (df["Adj Close"] / df["Adj Close"].shift(1)).apply(np.log)
df_test_log_ret = (df_test_1["Adj Close"] / df_test_1["Adj Close"].shift(1)).apply(np.log)

market_log_ret = (market["Adj Close"] / market["Adj Close"].shift(1)).apply(np.log)
market_test_log_ret = (market_test_1["Adj Close"] / market_test_1["Adj Close"].shift(1)).apply(np.log);

## Asset analysis

In [ ]:
legend_elements = []

i = 0

for asset in df["Adj Close"].columns
   
    c = "C$(i)"
        
    append!(legend_elements, [L2D([0], [0], color=c, lw=4, label=asset)])
    
    i += 1
    
end

fig, ax = plt.subplot_mosaic("""AB
    CD
    EF""", figsize=(8*2, 6*3))

df_total["Adj Close"].plot(ax=ax["A"], lw=3, legend=false)

ax["A"].set_yscale("log")

ax["A"].set_ylabel("Stock price", labelpad=10)

ax["A"].legend(handles=legend_elements, ncol=5, loc="upper center", bbox_to_anchor=(1.15, 1.55), fontsize=24)

#Histogram of returns
for asset in assets
    df_total["Adj Close"][asset].pct_change().hist(ax=ax["B"], bins=30, alpha=0.8, density=true, label=asset,
        legend=false, grid=false)
end

ax["B"].set_xlim(-0.1, 0.1)
ax["B"].set_xticks(np.arange(-0.1, 0.104, 0.04))

ax["B"].set_xlabel("Returns")
ax["B"].set_ylabel("Probability", labelpad=10)

rets = zeros(length(assets))
risks = zeros(length(assets))
skews = zeros(length(assets))
kurts = zeros(length(assets))

T = 252

for i in 1:length(assets)

    returns = df_total["Adj Close"][assets[i]].pct_change().dropna().values

    rets[i] =  100 * ((1+mean(returns))^T - 1)
    risks[i] = 100 * sqrt((var(returns)+ (1+mean(returns))^2)^T - (1+mean(returns))^(T*2))
    skews[i] = skewness(returns)
    kurts[i] = kurtosis(returns) + 3

end

idxs_ret = sortperm(rets, rev=true)
idxs_risk = sortperm(risks, rev=true)
idxs_skew = sortperm(skews, rev=true)
idxs_kurt = sortperm(kurts, rev=true)

ax["C"].bar(1:10, rets[idxs_ret])

ax["C"].set_xticks(1:10, assets[idxs_ret], rotation = 45)

ax["C"].set_ylabel("Annualized Returns [%]", labelpad=20)

ax["D"].bar(1:10, risks[idxs_risk])

ax["D"].set_xticks(1:10, assets[idxs_risk], rotation = 45)

ax["D"].set_ylabel("Annualized Volatility [%]", labelpad=20)

ax["E"].bar(1:10, skews[idxs_skew])

ax["E"].set_xticks(1:10, assets[idxs_skew], rotation = 45)

ax["E"].set_ylabel("Skewness", labelpad=20)

ax["F"].bar(1:10, kurts[idxs_kurt])

ax["F"].set_xticks(1:10, assets[idxs_kurt], rotation = 45)

ax["F"].set_ylabel("Kurtosis", labelpad=20)

#Labelling
for item in ax
    
    label, ax_i = item
    
    if (label == "A") | (label == "B")
    
    # label physical distance in and down:
    trans = matplotlib.transforms.ScaledTranslation(10/72, -15/72, fig.dpi_scale_trans)
        
    else
        
        trans = matplotlib.transforms.ScaledTranslation(4.6, -15/72, fig.dpi_scale_trans)
        
    end
    
    ax_i.text(0.0, 1.0, string("(", label, ")"), fontsize=30, transform=ax_i.transAxes + trans, 
        verticalalignment="top")
    
end

plt.subplots_adjust(hspace=0.4, wspace=0.3)

plt.savefig("Dataset_1_analysis.png", dpi=300, bbox_inches="tight")

In [ ]:
assets_Fortune_aux = ["WMT", "XOM", "F", "GE", "IBM", "C", "ATT", "MO", "BA", "BAC", "HPQ", "KR", "AIG", "HD", "PG"]

df_Fortune_aux = df_Fortune["Adj Close"]

df_Fortune_aux.columns = assets_Fortune_aux

colors = matplotlib.cm.get_cmap("tab20_r", 15)

legend_elements = []

i = 0

for asset in df_Fortune["Adj Close"].columns
   
    c = colors(i)
        
    append!(legend_elements, [L2D([0], [0], color=c, lw=4, label=asset)])
    
    i += 1
    
end

fig, ax = plt.subplot_mosaic("""AB
    CD
    EF""", figsize=(8*2, 6*3))

df_Fortune["Adj Close"].plot(ax=ax["A"], lw=3, legend=false, cmap=colors)

ax["A"].set_yscale("log")

ax["A"].set_ylabel("Stock price", labelpad=10)

ax["A"].legend(handles=legend_elements, ncol=5, loc="upper center", bbox_to_anchor=(1.15, 1.7), fontsize=24)

#Histogram of returns
i=0
for asset in assets_Fortune_aux
    df_Fortune_aux[asset].pct_change().hist(ax=ax["B"], bins=30, alpha=0.8, density=true, label=asset,
        legend=false, grid=false, color=colors(i))
    i+=1
end

ax["B"].set_xlim(-0.1, 0.1)
ax["B"].set_xticks(np.arange(-0.1, 0.104, 0.04))

ax["B"].set_xlabel("Returns")
ax["B"].set_ylabel("Probability", labelpad=10)

rets = zeros(length(assets_Fortune))
risks = zeros(length(assets_Fortune))
skews = zeros(length(assets_Fortune))
kurts = zeros(length(assets_Fortune))

T = 252

for i in 1:length(assets_Fortune)

    returns = df_Fortune_aux[assets_Fortune_aux[i]].pct_change().dropna().values

    rets[i] =  100 * ((1+mean(returns))^T - 1)
    risks[i] = 100 * sqrt((var(returns)+ (1+mean(returns))^2)^T - (1+mean(returns))^(T*2))
    skews[i] = skewness(returns)
    kurts[i] = kurtosis(returns) + 3

end


idxs_ret = sortperm(rets, rev=true)
idxs_risk = sortperm(risks, rev=true)
idxs_skew = sortperm(skews, rev=true)
idxs_kurt = sortperm(kurts, rev=true)

ax["C"].bar(1:15, rets[idxs_ret])

ax["C"].set_xticks(1:15, assets_Fortune[idxs_ret], rotation = 45)

ax["C"].set_ylabel("Annualized Returns [%]", labelpad=20)

ax["D"].bar(1:15, risks[idxs_risk])

ax["D"].set_xticks(1:15, assets_Fortune[idxs_risk], rotation = 45)

ax["D"].set_ylabel("Annualized Volatility [%]", labelpad=20)

ax["E"].bar(1:15, skews[idxs_skew])

ax["E"].set_xticks(1:15, assets_Fortune[idxs_skew], rotation = 45)

ax["E"].set_ylabel("Skewness", labelpad=20)

ax["F"].bar(1:15, kurts[idxs_kurt])

ax["F"].set_xticks(1:15, assets_Fortune[idxs_kurt], rotation = 45)

ax["F"].set_ylabel("Kurtosis", labelpad=20)

#Labelling
for item in ax
    
    label, ax_i = item
    
    if (label == "C") | (label == "D") | (label == "E") | (label == "F") | (label == "C")
        
        # label physical distance in and down:
        trans = matplotlib.transforms.ScaledTranslation(4.6, -15/72, fig.dpi_scale_trans)
        
    else
    
        # label physical distance in and down:
        trans = matplotlib.transforms.ScaledTranslation(10/72, -15/72, fig.dpi_scale_trans)
        
    end
        
    ax_i.text(0.0, 1.0, string("(", label, ")"), fontsize=30, transform=ax_i.transAxes + trans, 
        verticalalignment="top")
    
end

plt.subplots_adjust(hspace=0.4, wspace=0.3)

plt.savefig("Dataset_2_analysis.png", dpi=300, bbox_inches="tight")

# Resample data to 1-month frequency

In [ ]:
freq = "1M"

df = df.resample(freq).mean()

df_test_1 = df_test_1.resample(freq).mean()
df_test_2 = df_test_2.resample(freq).mean()
df_test_3 = df_test_3.resample(freq).mean()

market = market.resample(freq).mean()

market_test_1 = market_test_1.resample(freq).mean()
market_test_2 = market_test_2.resample(freq).mean()
market_test_3 = market_test_3.resample(freq).mean()

df_log_ret = (df["Adj Close"] / df["Adj Close"].shift(1)).apply(np.log)
df_test_1_log_ret = (df_test_1["Adj Close"] / df_test_1["Adj Close"].shift(1)).apply(np.log)
df_test_2_log_ret = (df_test_2["Adj Close"] / df_test_2["Adj Close"].shift(1)).apply(np.log)
df_test_3_log_ret = (df_test_3["Adj Close"] / df_test_3["Adj Close"].shift(1)).apply(np.log)

market_log_ret = (market["Adj Close"] / market["Adj Close"].shift(1)).apply(np.log)
market_test_log_ret = (market_test_1["Adj Close"] / market_test_1["Adj Close"].shift(1)).apply(np.log);

# Initialize portfolios 

In [ ]:
returns = df["Adj Close"].pct_change().dropna().values
log_returns = df_log_ret.dropna().values

returns_test_1 = df_test_1["Adj Close"].pct_change().dropna().values
returns_test_2 = df_test_2["Adj Close"].pct_change().dropna().values
returns_test_3 = df_test_3["Adj Close"].pct_change().dropna().values

log_returns_test_1 = df_test_1_log_ret.dropna().values
log_returns_test_2 = df_test_2_log_ret.dropna().values
log_returns_test_3 = df_test_3_log_ret.dropna().values

assets = [item for item in df["Adj Close"].columns]

portfolio = create_portfolio(returns, assets)
portfolio_log = create_portfolio(log_returns, assets)

portfolio_test_1 = create_portfolio(returns_test_1, assets)
portfolio_test_2 = create_portfolio(returns_test_2, assets)
portfolio_test_3 = create_portfolio(returns_test_3, assets)

portfolio_log_test_1 = create_portfolio(log_returns_test_1, assets)
portfolio_log_test_2 = create_portfolio(log_returns_test_2, assets)
portfolio_log_test_3 = create_portfolio(log_returns_test_3, assets);

# Mean-Variance portfolio optimization 

### Fixed returns 

In [ ]:
target_return = 0.03

return_p, risk_p, w_opt = MV_fixed_return(portfolio, target_return; method="DCP")

return_market = market["Adj Close"].pct_change().mean()
risk_market = sqrt(market["Adj Close"].pct_change().var())

println("Expected Returns Portfolio: ", return_p)
println("Risk Portfolio: ", risk_p)
#println("Weights:", w_opt)

println("\nExpected Return Market: ", return_market)
println("Risk Market: ", risk_market)

println("\nPortfolio-market return ratio: ", return_p/return_market)
println("Portfolio-market risk ratio: ", risk_p/risk_market)

### Fixed risk 

In [ ]:
target_risk = 0.035

return_p, risk_p, w_opt = MV_fixed_risk(portfolio, target_risk; method="DCP")

return_market = market["Adj Close"].pct_change().mean()
risk_market = sqrt(market["Adj Close"].pct_change().var())

println("Expected Returns Portfolio: ", return_p)
println("Risk Portfolio: ", risk_p)
#println("Weights:", w_opt)

println("\nExpected Return Market: ", return_market)
println("Risk Market: ", risk_market)

println("\nPortfolio-market return ratio: ", return_p/return_market)
println("Portfolio-market risk ratio: ", risk_p/risk_market)

# Efficient frontier 

In [ ]:
RetRisk, weights = MV_efficient_frontier(portfolio, 100);

RetRisk_log, weights_log = MV_efficient_frontier(portfolio_log, 100);

risks = sqrt.(diag(portfolio.Σ))
risks_log = sqrt.(diag(portfolio_log.Σ))

##### FIGURE ######

legend_elements = []

i = 0

for asset in portfolio.assets
   
    c = "C$(i)"
        
    append!(legend_elements, [L2D([0], [0], color=c, lw=4, label=asset)])
    
    i += 1
    
end

append!(legend_elements, [L2D([0], [0], ls="", marker="d", color="k", markersize=12, label="S&P500")])
append!(legend_elements, [L2D([0], [0], ls="--", lw=3, color="k", label="Efficient frontier")])


fig, ax = plt.subplot_mosaic("""AB
    CD""", figsize=(8*2, 6*2))

for i in 1:length(portfolio.μ)

    ax["A"].scatter(risks[i], portfolio.μ[i], s=150)
    
end

ax["A"].scatter(risk_market, return_market, s=150, color="k", marker="d", label="S&P500")

ax["A"].plot(RetRisk[:, 2], RetRisk[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

ax["A"].set_ylabel("Return", labelpad=15)
ax["A"].set_xlabel("Volatility", labelpad=15)

ax["A"].legend(handles=legend_elements, ncol=6, loc="upper center", bbox_to_anchor=(1.05, 1.4))

for i in 1:length(portfolio.μ)

    ax["B"].scatter(risks_log[i], portfolio_log.μ[i], s=150)
    
end

ax["B"].scatter(risk_market, return_market, s=150, color="k", marker="d", label="S&P500")

ax["B"].plot(RetRisk_log[:, 2], RetRisk_log[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

ax["B"].set_ylabel("Return", labelpad=15)
ax["B"].set_xlabel("Volatility", labelpad=15)

#plt.legend(ncol=2)

#
N = Int(10^6)

rets = zeros(N)
risks = zeros(N)
sharpe_ratio = zeros(N)

for i in 1 : N

    w = rand(length(portfolio.μ))

    w = w / sum(w)
   
    return_p = dot(w, portfolio.μ)
    risk_p = sqrt(transpose(w)*portfolio.Σ*w)
    
    rets[i] = return_p
    risks[i] = risk_p
    
    sharpe_ratio[i] = return_p / risk_p
    
end

ax["C"].plot(RetRisk[:, 2], RetRisk[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

cplot = ax["C"].scatter(risks, rets, c=sharpe_ratio, cmap="jet")

ax["C"].set_ylabel("Return", labelpad=15)
ax["C"].set_xlabel("Volatility", labelpad=15)

cbar = plt.colorbar(cplot, ax=ax["C"])

cbar.ax.set_title("Sharpe Ratio", fontsize=20)

#
N = Int(10^6)

rets = zeros(N)
risks = zeros(N)
sharpe_ratio = zeros(N)

for i in 1 : N

    w = rand(length(portfolio_log.μ))

    w = w / sum(w)
   
    return_p = dot(w, portfolio_log.μ)
    risk_p = sqrt(transpose(w)*portfolio_log.Σ*w)
    
    rets[i] = return_p
    risks[i] = risk_p
    
    sharpe_ratio[i] = return_p / risk_p
    
end

ax["D"].plot(RetRisk_log[:, 2], RetRisk_log[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

cplot = ax["D"].scatter(risks, rets, c=sharpe_ratio, cmap="jet")

ax["D"].set_ylabel("Log return", labelpad=15)
ax["D"].set_xlabel("Volatility", labelpad=15)

cbar = plt.colorbar(cplot, ax=ax["D"])

cbar.ax.set_title("Sharpe Ratio", fontsize=20)

#Labelling
for item in ax
    
    label, ax_i = item
    
    # label physical distance in and down:
    trans = matplotlib.transforms.ScaledTranslation(10/72, -15/72, fig.dpi_scale_trans)
    ax_i.text(0.0, 1.0, string("(", label, ")"), fontsize=30, transform=ax_i.transAxes + trans, 
        verticalalignment="top")
    
end

plt.subplots_adjust(wspace=0.3, hspace=0.35)

plt.savefig("Efficient_frontier_MV.png", bbox_inches="tight", dpi=300)

In [ ]:
plt.plot(RetRisk[:, 2], RetRisk[:, 1], color="green", lw=4, ls="-", label="Efficient frontier")

plt.fill_between(RetRisk[:, 2], 0.016 * np.ones(length(RetRisk[:, 2])), RetRisk[:, 1], color="r", alpha=0.2)
plt.fill_between(RetRisk[:, 2], RetRisk[:, 1], 0.05 * np.ones(length(RetRisk[:, 2])), color="gray", alpha=0.2)

plt.text(0.037, 0.028, "Sub-optimal portfolios", color="r")
plt.text(0.025, 0.033, "Infeasible portfolios", rotation=30, color="gray")

plt.annotate("", (0.023, 0.020), (0.03, 0.020), arrowprops=Dict("arrowstyle"=>"->", "color"=>"k", "linewidth"=>2),)
plt.text(0.031, 0.02, L"$\lambda=1$ (risk minimization)", fontsize=16)

plt.annotate("", (0.0648, 0.047), (0.0605, 0.0405), arrowprops=Dict("arrowstyle"=>"->", "color"=>"k", "linewidth"=>2),)
plt.text(0.058, 0.039, L"$\lambda=0$", rotation=0, fontsize=16)
plt.text(0.047, 0.037, "(return maximization)", fontsize=16)

plt.xlabel(L"Portfolio Risk $\sigma_P$", labelpad=10)
plt.ylabel(L"Portfolio Return $\mu_P$", labelpad=10)

plt.xlim(0.0228,0.065)
plt.ylim(0.018, 0.05)

plt.legend()

plt.savefig("Efficient_frontier_example.png", bbox_inches="tight", dpi=300)

# Portfolio testing 

Up to this moment we have optimized the weights of our portfolio for any desired level of returns and risk. However, this is done in a training (in-sample) set, but in the future things could change and our optimal weights could'nt be optimal any more. Here we check how expected returns and expected risk for the optimized weights could vary in the future by applying our optimal weights to a test (out of sample) set. We will show 2 results:

- An example of the out-of-sample performance of the in-sample optimized portfolio yielding a fixed return of 0.025

- The Yield Curve of all in-sample optimal portfolios (the ones on the efficient frontier) in the out-of-sample set.

In [ ]:
#EXAMPLE

#Test in "future" data
#df_test_returns = df_test["Adj Close"].pct_change().dropna().values

target_return = 0.025

#Optimize in past data
ret_opt, risk_opt, w_opt = MV_fixed_return(portfolio, target_return)

returns_p_opt_test_1 = returns_test_1 * w_opt
returns_p_opt_test_2 = returns_test_2 * w_opt
returns_p_opt_test_3 = returns_test_3 * w_opt
    
risk = round(sqrt(var(returns_test_1)), digits=3)

dates_1 = [df_test_1["Adj Close"].index[i] for i in 2: length(df_test_1)]
dates_2 = [df_test_2["Adj Close"].index[i] for i in 2: length(df_test_2)]
dates_3 = [df_test_3["Adj Close"].index[i] for i in 2: length(df_test_3)]

#mean_return = round(mean(returns_test), digits=4)
#return_market = round(market_test["Adj Close"].pct_change().mean(), digits=4)
#risk_market = round(sqrt(market_test["Adj Close"].pct_change().var()), digits=3);

target_returns = 0.021 : 0.002 : 0.048

final_risks_1 = []
final_returns_1 = []

final_risks_2 = []
final_returns_2 = []

final_risks_3 = []
final_returns_3 = []

expected_risks = []
expected_returns = []

for target_return in target_returns

    expected_return, expected_risk, w_opt = MV_fixed_return(portfolio, target_return)

    returns_1 = returns_test_1 * w_opt
    returns_2 = returns_test_2 * w_opt
    returns_3 = returns_test_3 * w_opt

    actual_risk_1 = round(sqrt(var(returns_1)), digits=6)
    actual_mean_return_1 = round(mean(returns_1), digits=6)
    
    actual_risk_2 = round(sqrt(var(returns_2)), digits=6)
    actual_mean_return_2 = round(mean(returns_2), digits=6)
    
    actual_risk_3 = round(sqrt(var(returns_3)), digits=6)
    actual_mean_return_3 = round(mean(returns_3), digits=6)
    
    append!(expected_risks, expected_risk)
    append!(expected_returns, expected_return)
    
    append!(final_risks_1, actual_risk_1)
    append!(final_returns_1, actual_mean_return_1)
    
    append!(final_risks_2, actual_risk_2)
    append!(final_returns_2, actual_mean_return_2)
    
    append!(final_risks_3, actual_risk_3)
    append!(final_returns_3, actual_mean_return_3)
    
end

RetRisk_test_1, weights_1 = MV_efficient_frontier(portfolio_test_1, 50)
RetRisk_test_2, weights_2 = MV_efficient_frontier(portfolio_test_2, 25)
RetRisk_test_3, weights_3 = MV_efficient_frontier(portfolio_test_3, 25);

In [ ]:
fig, ax = plt.subplot_mosaic("AB", figsize=(8*2,6))

#ax["A"].plot(dates_1, mean(returns_test_1, dims=2), marker="o", ms=8, label="Portfolio simulated returns")
#ax["A"].plot(dates_2, mean(returns_test_2, dims=2), marker="o", ms=8, label="Portfolio simulated returns")
ax["A"].plot(dates_3, mean(returns_test_3, dims=2), marker="o", ms=8, label="Portfolio simulated returns")

ax["A"].plot(dates_1, [-0.06 for item in dates_1], color="g", lw=3, label="Test 1")
ax["A"].plot(dates_2, [-0.07 for item in dates_2], lw=3, color="orange", label="Test 1")
ax["A"].plot(dates_3, [-0.08 for item in dates_3], lw=3, color="r", label="Test 3")

#ax["A"].axhline(mean_return, color="k", lw=3, label="Portfolio: μ=$mean_return,  σ=$risk")
#ax["A"].axhline(return_market, color="k", lw=3, ls="--", label="Market: μ=$return_market,  σ=$risk_market")

ax["A"].tick_params("x", rotation=45)

ax["A"].set_ylim(-0.0885, 0.12)

ax["A"].legend(loc="upper center",ncol=2, bbox_to_anchor=(0.5, 1.3))

#ax["B"].scatter(risk_market, return_market, color="k", s=100, marker="d", label="S&P500")

ax["B"].plot(expected_risks, expected_returns, color="k", lw=5)

ax["B"].plot(final_risks_1, final_returns_1, marker="o", c="C0", ms=8)
ax["B"].plot(final_risks_2, final_returns_2, marker="^", c="C0", ms=8)
ax["B"].plot(final_risks_3, final_returns_3, marker="s", c="C0", ms=8)

ax["B"].plot(RetRisk_test_1[:, 2], RetRisk_test_1[:, 1], color="C1", ls="-", marker="o", ms=8, 
    label="Test 1 Efficient Frontier")
ax["B"].plot(RetRisk_test_2[:, 2], RetRisk_test_2[:, 1], color="C1", ls="-", marker="^", ms=8, 
    label="Test 2 Efficient Frontier")
ax["B"].plot(RetRisk_test_3[:, 2], RetRisk_test_3[:, 1], color="C1", ls="-", marker="s", ms=8, 
    label="Test 3 Efficient Frontier")

ax["B"].set_ylabel("Returns", fontsize=20, labelpad=15)
ax["B"].set_xlabel("Risk", fontsize=20, labelpad=15)

legend_elements = [L2D([0], [0], lw=5, color="k", label="In-sample efficient frontier"),
                   L2D([0], [0], lw=5, color="C1", label="Test efficient frontier"),
                   L2D([0], [0], lw=5, color="C0", label="Test portfolio performance"),
                   L2D([0], [0], ls="", marker="o", color="k", ms=10, label="Test 1"),
                   L2D([0], [0], ls="", marker="^", color="k", ms=10, label="Test 2"),
                   L2D([0], [0], ls="", marker="s", color="k", ms=10, label="Test 3")]

ax["B"].legend(handles=legend_elements, loc="upper center", ncol=2, bbox_to_anchor=(0.5, 1.35))

#Labelling
for item in ax
    
    label, ax_i = item
    
    # label physical distance in and down:
    trans = matplotlib.transforms.ScaledTranslation(10/72, -15/72, fig.dpi_scale_trans)
    ax_i.text(0.0, 1.0, string("(", label, ")"), fontsize=30, transform=ax_i.transAxes + trans, 
        verticalalignment="top")
    
end

plt.subplots_adjust(wspace=0.3)

- The in-sample optimized portfolios are always suboptimal in the testing period (which could be expected). However, we observe that if the testing period is near the training period (so if it shprt enough) the in-sample optimized portfolios are a good approximation to the test efficient frontier.

- It is interesting to observe that, for the near-future case, the in-sample optimized portfolios indeed outperform the expected results from the in-sample efficient frontier

- We also note that the efficient frontiers of the 2nd and rd testing periods are quite different from that of the 1st period, showing that the optimization problem is harder. This is probably because the high volatility in those periods.

# Portfolio Evolutionary Optimization 

## Sanity check with MV model 

### Fixed return optimization 

In [ ]:
target_return = 0.03

return_p_DCP, risk_p_DCP, w_opt_DCP = MV_fixed_return(portfolio, target_return; method="DCP")
return_p_EO, risk_p_EO, w_opt_EO = MV_fixed_return(portfolio, target_return; method="EO")

println("Expected Returns Portfolio (DCP): ", return_p_DCP)
println("Expected Returns Portfolio (EO): ", return_p_EO)

println("\nRisk Portfolio (DCP): ", risk_p_DCP)
println("Risk Portfolio (EO): ", risk_p_EO)

### Fixed risk optimization 

In [ ]:
target_risk = 0.035

return_p_DCP, risk_p_DCP, w_opt_DCP = MV_fixed_risk(portfolio, target_risk; method="DCP")
return_p_EO, risk_p_EO, w_opt_EO = MV_fixed_risk(portfolio, target_risk; method="EO")

println("Expected Returns Portfolio (DCP): ", return_p_DCP)
println("Expected Returns Portfolio (EO): ", return_p_EO)

println("\nRisk Portfolio (DCP): ", risk_p_DCP)
println("Risk Portfolio (EO): ", risk_p_EO)

### Efficient Frontier 

In [ ]:
#Efficient frontier using Convex Optimization
@time RetRisk_DCP, weights_DCP = MV_efficient_frontier(portfolio, 50; method="DCP")

#Efficient frontier using Evolutionary Optimization
@time RetRisk_EO, weights_EO = MV_efficient_frontier(portfolio, 30; method="EO");

In [ ]:
plt.plot(RetRisk_DCP[:, 2], RetRisk_DCP[:, 1], color="g", lw=5, zorder=1, label="DCP")
plt.scatter(RetRisk_EO[:, 2], RetRisk_EO[:, 1], color="r", label="EO", s=120)

#plt.xlabel(L"$\sigma_P$")
#plt.ylabel(L"$\mu_P$")
plt.xlabel("Volatility", labelpad=10)
plt.ylabel("Return", labelpad=10)

plt.legend()

plt.savefig("DCP_EO_check.png", dpi=300, bbox_inches="tight")

# Higher-Moment Portfolio Optimization 

Now the optimization problem is non-convex, so DCP can not be used. We will optimize the portfolios under MVSK model using Evolutionary Optimization.

## High-dimensional efficient frontier 

This optimization problem is highly computationally demanding, so we run the simulations in a cluster. Here we load the data and show some results.

In [ ]:
#Generate random portfolios anc compute M-V-S-K
N = Int(10^6)

rets = zeros(N)
risks = zeros(N)
skews = zeros(N)
kurts = zeros(N)

sharpe_ratio = zeros(N)

@time for i in 1 : N

    w = rand(length(portfolio.μ))

    w = w / sum(w)
   
    return_p = dot(w, portfolio.μ)
    risk_p = sqrt(transpose(w)*portfolio.Σ*w)
    skew_p = standarized_skewness_portfolio(portfolio, w)
    kurt_p = standarized_excess_kurtosis_portfolio(portfolio, w)
    
    rets[i] = return_p
    risks[i] = risk_p
    skews[i] = skew_p
    kurts[i] = kurt_p
    
    sharpe_ratio[i] = return_p / risk_p
    
end

RetRisk_DCP, weights_DCP = MV_efficient_frontier(portfolio, 50; method="DCP")

#Load optimized portfolios
filenames = readdir("HD_efficient_frontier")

results = zeros((length(filenames), 4))
λs = zeros((length(filenames), 4))
Weights = zeros((length(filenames), 10))

i = 1

for filename in filenames
   
    data, header = readdlm(string("HD_efficient_frontier/", filename), header=true)
   
    results[i, :] = data[1:4]
    λs[i, :] = data[5:8]
    Weights[i, :] = data[9:end]
    
    i += 1
    
end

results_MV = sort(results[(λs[:, 3] .== 0.0) .& (λs[:, 4] .== 0.0), :], dims=1)
results_MK = sort(results[(λs[:, 2] .== 0.0) .& (λs[:, 3] .== 0.0), :], dims=1)
results_SK = sort(results[(λs[:, 1] .== 0.0) .& (λs[:, 2] .== 0.0), :], dims=1);

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
    CD""", figsize=(8*2, 6*2))

cplot = ax["A"].scatter(kurts, skews, c=rets, cmap="jet")
ax["A"].plot(results_SK[:, 4], results_SK[:, 3], color="k", lw=3, ls="--")

ax["A"].set_xlabel("Kurtosis")
ax["A"].set_ylabel("Skewness")

#ax["A"].set_xlim(1e-7, 5e-4)
#ax["A"].set_ylim(-6e-5, 5e-4)

#ax["A"].set_yscale("log")
#ax["A"].set_xscale("log")

cbar = plt.colorbar(cplot, ax=ax["A"])

cbar.ax.set_title("Return", fontsize=20)

cplot = ax["B"].scatter(kurts, skews, c=risks, cmap="jet")
ax["B"].plot(results_SK[:, 4], results_SK[:, 3], color="k", lw=3, ls="--")

ax["B"].set_xlabel("Kurtosis")
ax["B"].set_ylabel("Skewness")

#ax["B"].set_xlim(1e-7, 5e-4)
#ax["B"].set_ylim(-6e-5, 5e-4)

#ax["B"].set_xscale("log")

cbar = plt.colorbar(cplot, ax=ax["B"])

cbar.ax.set_title("Volatility", fontsize=20)

cplot = ax["C"].scatter(risks, rets, c=skews, cmap="jet")
ax["C"].plot(results_MV[:, 2], results_MV[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

ax["C"].set_xlabel("Volatility")
ax["C"].set_ylabel("Return")

cbar = plt.colorbar(cplot, ax=ax["C"])

cbar.ax.set_title("Skewness", fontsize=20)

cplot = ax["D"].scatter(risks, rets, c=kurts, cmap="jet")
ax["D"].plot(results_MV[:, 2], results_MV[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

ax["D"].set_xlabel("Volatility")
ax["D"].set_ylabel("Return")

cbar = plt.colorbar(cplot, ax=ax["D"])

cbar.ax.set_title("Kurtosis", fontsize=20)

#Labelling
for item in ax
    
    label, ax_i = item
    
    # label physical distance in and down:
    trans = matplotlib.transforms.ScaledTranslation(10/72, -15/72, fig.dpi_scale_trans)
    ax_i.text(0.0, 1.0, string("(", label, ")"), fontsize=30, transform=ax_i.transAxes + trans, 
        verticalalignment="top")
    
end

plt.subplots_adjust(wspace=0.4, hspace=0.25)

plt.savefig("Efficient_frontier_MVSK.png", bbox_inches="tight", dpi=300)

## In-Sample Portfolio Optimization

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
CD""", figsize=(8*2,6*2))

cn1 = ax["A"].scatter(results[:, 2], results[:, 1], c=results[:, 3], s=60, cmap="Set2")

ax["A"].plot(results_MV[:, 2], results_MV[:, 1], lw=3, color="k", ls="-", label="M-V")
ax["A"].plot(results_SK[:, 2], results_SK[:, 1], lw=3, color="k", ls=":", label="S-K")

ax["A"].legend(loc="upper center", ncol=3, bbox_to_anchor=(1.25, 1.3))

cbar1 = fig.colorbar(cn1, ax=ax["A"])
cbar1.ax.set_title("Skewness", pad=10, fontsize=16)

ax["A"].set_xlabel("Volatility")
ax["A"].set_ylabel("Return")

cn2 = ax["B"].scatter(results[:, 2], results[:, 1], c=results[:, 4], s=60, cmap="Set2")

cbar2 = fig.colorbar(cn2, ax=ax["B"])
cbar2.ax.set_title("Kurtosis", pad=10, fontsize=16)

ax["B"].plot(results_MV[:, 2], results_MV[:, 1], lw=3, color="k", ls="-", label="M-V")
ax["B"].plot(results_SK[:, 2], results_SK[:, 1], lw=3, color="k", ls=":", label="S-K")

ax["B"].set_xlabel("Volatility")
ax["B"].set_ylabel("Return")

cn3 = ax["C"].scatter(results[:, 4], results[:, 3], c=results[:, 1], s=60, cmap="Set2")

cbar3 = fig.colorbar(cn3, ax=ax["C"])
cbar3.ax.set_title("Return", pad=10, fontsize=16)

ax["C"].plot(results_MV[:, 4], results_MV[:, 3], lw=3, color="k", ls="-", label="M-V")
ax["C"].plot(results_SK[:, 4], results_SK[:, 3], lw=3, color="k", ls=":", label="S-K")

#=
axinsC = ax["C"].inset_axes([0.2, 0.3, 0.5, 0.5])

axinsC.scatter(results[:, 4], results[:, 3], c=results[:, 1], s=60, cmap="Set2")
axinsC.plot(results_MV[:, 4], results_MV[:, 3], lw=3, color="k", ls="-", label="M-V")
axinsC.plot(results_SK[:, 4], results_SK[:, 3], lw=3, color="k", ls=":", label="S-K")

axinsC.set_xlim(1e-7, 5e-4)
axinsC.set_ylim(-6e-5, 5e-4)

axinsC.set_xscale("log")

axinsC.set_xticks([])
axinsC.set_yticks([])

ax["C"].indicate_inset_zoom(axinsC, edgecolor="black")
=#

ax["C"].set_xlabel("Kurtosis")
ax["C"].set_ylabel("Skewness")

#ax["C"].set_xlim(1e-7, 5e-4)
#ax["C"].set_ylim(-6e-5, 5e-4)

cn4 = ax["D"].scatter(results[:, 4], results[:, 3], c=results[:, 2], s=60, cmap="Set2")

cbar4 = fig.colorbar(cn4, ax=ax["D"])
cbar4.ax.set_title("Volatility", pad=10, fontsize=16)

ax["D"].plot(results_MV[:, 4], results_MV[:, 3], lw=3, color="k", ls="-", label="M-V")
ax["D"].plot(results_SK[:, 4], results_SK[:, 3], lw=3, color="k", ls=":", label="S-K")

#=
axinsD = ax["D"].inset_axes([0.1, 0.3, 0.5, 0.5])

axinsD.scatter(results[:, 4], results[:, 3], c=results[:, 2], s=60, cmap="Set2")

axinsD.plot(results_MV[:, 4], results_MV[:, 3], lw=3, color="k", ls="-", label="M-V")
axinsD.plot(results_SK[:, 4], results_SK[:, 3], lw=3, color="k", ls=":", label="S-K")

axinsD.set_xscale("log")

axinsD.set_xlim(5e-7, 1e-3)
axinsD.set_ylim(-6e-5, 5e-4)

axinsD.set_xticks([])
axinsD.set_yticks([])

ax["D"].indicate_inset_zoom(axinsD, edgecolor="black")
=#

ax["D"].set_xlabel("Kurtosis")
ax["D"].set_ylabel("Skewness")

#Labelling
for item in ax
    
    label, ax_i = item
    
    # label physical distance in and down:
    trans = matplotlib.transforms.ScaledTranslation(10/72, -15/72, fig.dpi_scale_trans)
    ax_i.text(0.0, 1.0, string("(", label, ")"), fontsize=30, transform=ax_i.transAxes + trans, 
        verticalalignment="top")
    
end

plt.subplots_adjust(wspace=0.25, hspace=0.25)

plt.savefig("In_sample_MVSK.png", bbox_inches="tight", dpi=300)

In [ ]:
fig = plt.figure(figsize = (8*2, 6))

ax = plt.subplot(1, 2, 1, projection ="3d")

cnplot = ax.scatter(results[:, 1], results[:, 2], results[:, 3], c=results[:, 4].+3, cmap="jet", marker="o")

ax.zaxis.set_rotate_label(false)

ax.set_xlabel(L"$\mu_P$", labelpad=12)
ax.set_ylabel(L"$\sigma_P$", labelpad=20)
ax.set_zlabel(L"$S_P$", labelpad=12)

ax = plt.subplot(1, 2, 2, projection ="3d")

cnplot = ax.scatter(results[:, 1], results[:, 2], results[:, 3], c=results[:, 4].+3, cmap="jet", marker="o")


ax.zaxis.set_rotate_label(false)
ax.xaxis.set_rotate_label(false)

ax.set_xlabel(L"$\mu_P$", labelpad=30)
ax.set_ylabel(L"$\sigma_P$", labelpad=12)
ax.set_zlabel(L"$S_P$", labelpad=20)

ax.view_init(azim=0, elev=30)

plt.subplots_adjust(wspace=0.0)

plt.savefig("4D_efficient_frontier.png", bbox_inches="tight", dpi=300)

# Out of sample Higher Moment Portfolio Optimization 

In [ ]:
#Be careful, StatsBase.jl moment() function computes the standarized moment!

N_portfolios = size(Weights)[1]

results_p_opt_1 = zeros((N_portfolios, 4))
results_p_opt_2 = zeros((N_portfolios, 4))
results_p_opt_3 = zeros((N_portfolios, 4))

for i in 1 : N_portfolios
    
    res_i_1 = returns_test_1 * Weights[i, :]
    res_i_2 = returns_test_2 * Weights[i, :]
    res_i_3 = returns_test_3 * Weights[i, :]
    
    results_p_opt_1[i, :] = [mean(res_i_1), sqrt(var(res_i_1)), skewness(res_i_1), kurtosis(res_i_1)]
    results_p_opt_2[i, :] = [mean(res_i_2), sqrt(var(res_i_2)), skewness(res_i_2), kurtosis(res_i_2)]
    results_p_opt_3[i, :] = [mean(res_i_3), sqrt(var(res_i_3)), skewness(res_i_3), kurtosis(res_i_3)]
        
end

results_MV_OOS_1 = sort(results_p_opt_1[(λs[:, 3] .== 0.0) .& (λs[:, 4] .== 0.0), :], dims=1)
results_MK_OOS_1 = sort(results_p_opt_1[(λs[:, 2] .== 0.0) .& (λs[:, 3] .== 0.0), :], dims=1)
results_SK_OOS_1 = sort(results_p_opt_1[(λs[:, 1] .== 0.0) .& (λs[:, 2] .== 0.0), :], dims=1)

results_MV_OOS_2 = sort(results_p_opt_2[(λs[:, 3] .== 0.0) .& (λs[:, 4] .== 0.0), :], dims=1)
results_MK_OOS_2 = sort(results_p_opt_2[(λs[:, 2] .== 0.0) .& (λs[:, 3] .== 0.0), :], dims=1)
results_SK_OOS_2 = sort(results_p_opt_2[(λs[:, 1] .== 0.0) .& (λs[:, 2] .== 0.0), :], dims=1)

results_MV_OOS_3 = sort(results_p_opt_3[(λs[:, 3] .== 0.0) .& (λs[:, 4] .== 0.0), :], dims=1)
results_MK_OOS_3 = sort(results_p_opt_3[(λs[:, 2] .== 0.0) .& (λs[:, 3] .== 0.0), :], dims=1)
results_SK_OOS_3 = sort(results_p_opt_3[(λs[:, 1] .== 0.0) .& (λs[:, 2] .== 0.0), :], dims=1);

#Load optimized portfolios in the Out of sample period
filenames_OOS_1 = readdir("HD_efficient_frontier_test_1")
filenames_OOS_2 = readdir("HD_efficient_frontier_test_2")
filenames_OOS_3 = readdir("HD_efficient_frontier_test_3")

results_OOS_1 = zeros((length(filenames), 4))
results_OOS_2 = zeros((length(filenames), 4))
results_OOS_3 = zeros((length(filenames), 4))

λs_test_1 = zeros((length(filenames_OOS_1), 4))
λs_test_2 = zeros((length(filenames_OOS_2), 4))
λs_test_3 = zeros((length(filenames_OOS_3), 4))

i = 1

for filename in filenames_OOS_1
   
    data, header = readdlm(string("HD_efficient_frontier_test_1/", filename), header=true)
   
    results_OOS_1[i, :] = data[1:4]
    
    λs_test_1[i, :] = data[5:8]
    
    i += 1
    
end

i = 1

for filename in filenames_OOS_1
   
    data, header = readdlm(string("HD_efficient_frontier_test_2/", filename), header=true)
   
    results_OOS_2[i, :] = data[1:4]
    
    λs_test_2[i, :] = data[5:8]
    
    i += 1
    
end

i = 1

for filename in filenames_OOS_1
   
    data, header = readdlm(string("HD_efficient_frontier_test_3/", filename), header=true)
   
    results_OOS_3[i, :] = data[1:4]
    
    λs_test_3[i, :] = data[5:8]
    
    i += 1
    
end

results_OOS_MV_opt_1 = sort(results_OOS_1[(λs_test_1[:, 3] .== 0.0) .& (λs_test_1[:, 4] .== 0.0), :], dims=1)
results_OOS_MK_opt_1 = sort(results_OOS_1[(λs_test_1[:, 2] .== 0.0) .& (λs_test_1[:, 3] .== 0.0), :], dims=1)
results_OOS_SK_opt_1 = sort(results_OOS_1[(λs_test_1[:, 1] .== 0.0) .& (λs_test_1[:, 2] .== 0.0), :], dims=1)

results_OOS_MV_opt_2 = sort(results_OOS_2[(λs_test_2[:, 3] .== 0.0) .& (λs_test_2[:, 4] .== 0.0), :], dims=1)
results_OOS_MK_opt_2 = sort(results_OOS_2[(λs_test_2[:, 2] .== 0.0) .& (λs_test_2[:, 3] .== 0.0), :], dims=1)
results_OOS_SK_opt_2 = sort(results_OOS_2[(λs_test_2[:, 1] .== 0.0) .& (λs_test_2[:, 2] .== 0.0), :], dims=1)

results_OOS_MV_opt_3 = sort(results_OOS_3[(λs_test_3[:, 3] .== 0.0) .& (λs_test_3[:, 4] .== 0.0), :], dims=1)
results_OOS_MK_opt_3 = sort(results_OOS_3[(λs_test_3[:, 2] .== 0.0) .& (λs_test_3[:, 3] .== 0.0), :], dims=1)
results_OOS_SK_opt_3 = sort(results_OOS_3[(λs_test_3[:, 1] .== 0.0) .& (λs_test_3[:, 2] .== 0.0), :], dims=1);

legend_elements = [L2D([0], [0], lw=5, color="k", label="In-sample efficient frontier"),
                   L2D([0], [0], lw=5, color="C1", label="Test efficient frontier"),
                   L2D([0], [0], lw=5, color="C0", label="Test portfolio performance"),
                   L2D([0], [0], ls="", marker="o", color="k", ms=10, label="Test 1"),
                   L2D([0], [0], ls="", marker="^", color="k", ms=10, label="Test 2"),
                   L2D([0], [0], ls="", marker="s", color="k", ms=10, label="Test 3")]

fig, ax = plt.subplot_mosaic("""AB
    CD""", figsize=(8*2,6*2))

ax["A"].plot(results_MV[:, 2], results_MV[:, 1], lw=5, color="k", label="M-V")

ax["A"].plot(results_OOS_MV_opt_1[:, 2], results_OOS_MV_opt_1[:, 1], color="C1", marker="o", ms=8)
ax["A"].plot(results_OOS_MV_opt_2[:, 2], results_OOS_MV_opt_2[:, 1], color="C1", marker="^", ms=8)
ax["A"].plot(results_OOS_MV_opt_3[:, 2], results_OOS_MV_opt_3[:, 1], color="C1", marker="s", ms=8)

ax["A"].plot(results_MV_OOS_1[:, 2], results_MV_OOS_1[:, 1], color="C0", ls="-", marker="o", ms=8)
ax["A"].plot(results_MV_OOS_2[:, 2], results_MV_OOS_2[:, 1], color="C0", ls="-", marker="^", ms=8)
ax["A"].plot(results_MV_OOS_3[:, 2], results_MV_OOS_3[:, 1], color="C0", ls="-", marker="s", ms=8)

ax["A"].set_xlabel("Volatility")
ax["A"].set_ylabel("Return")

ax["A"].set_title("M-V")

ax["A"].legend(handles=legend_elements, loc="upper center", ncol=3, bbox_to_anchor=(1.05, 1.45))

ax["B"].plot(results_SK[:, 4], results_SK[:, 3], lw=5, color="k", label="S-K")

ax["B"].plot(results_OOS_SK_opt_1[:, 4], results_OOS_SK_opt_1[:, 3], color="C1", marker="o", ms=8)
ax["B"].plot(results_OOS_SK_opt_2[:, 4], results_OOS_SK_opt_2[:, 3], color="C1", marker="^", ms=8)
ax["B"].plot(results_OOS_SK_opt_3[:, 4], results_OOS_SK_opt_3[:, 3], color="C1", marker="s", ms=8)

ax["B"].plot(results_SK_OOS_1[:, 4], results_SK_OOS_1[:, 3], color="C0", ls="-", marker="o", ms=8)
ax["B"].plot(results_SK_OOS_2[:, 4], results_SK_OOS_2[:, 3], color="C0", ls="-", marker="^", ms=8)
ax["B"].plot(results_SK_OOS_3[:, 4], results_SK_OOS_3[:, 3], color="C0", ls="-", marker="s", ms=8)

ax["B"].set_title("S-K")
ax["B"].set_xlabel("Kurtosis")
ax["B"].set_ylabel("Skewness")

ax["B"].set_xlim(-2.2, 2)
ax["B"].set_ylim(-0.6, 1.7)

MV_dist_1 = mean(sqrt.(sum((results_OOS_MV_opt_1[:, 1:2] .- results_MV_OOS_1[:, 1:2]).^2, dims=2)))
MV_dist_2 = mean(sqrt.(sum((results_OOS_MV_opt_2[:, 1:2] .- results_MV_OOS_2[:, 1:2]).^2, dims=2)))
MV_dist_3 = mean(sqrt.(sum((results_OOS_MV_opt_3[:, 1:2] .- results_MV_OOS_3[:, 1:2]).^2, dims=2)))

SK_dist_1 = mean(sqrt.(sum((results_OOS_MV_opt_1[:, 3:4] .- results_MV_OOS_1[:, 3:4]).^2, dims=2)))
SK_dist_2 = mean(sqrt.(sum((results_OOS_MV_opt_2[:, 3:4] .- results_MV_OOS_2[:, 3:4]).^2, dims=2)))
SK_dist_3 = mean(sqrt.(sum((results_OOS_MV_opt_3[:, 3:4] .- results_MV_OOS_3[:, 3:4]).^2, dims=2)))

ax["C"].bar(["Test 1", "Test 2", "Test 3"], [MV_dist_1, MV_dist_2, MV_dist_3], label="MV")

ax["C"].set_title("M-V")
ax["C"].set_ylabel("Distance to EF")

ax["D"].bar(["Test 1", "Test 2", "Test 3"], [SK_dist_1, SK_dist_2, SK_dist_3], label="SK")

ax["D"].set_title("S-K")
ax["D"].set_ylabel("Distance to EF")

ax["C"].tick_params("x", labelsize=20)
ax["D"].tick_params("x", labelsize=20)

#Labelling
for item in ax
    
    label, ax_i = item
    
    # label physical distance in and down:
    
    if label == "D"
        trans = matplotlib.transforms.ScaledTranslation(4.5, -15/72, fig.dpi_scale_trans)
    else
        trans = matplotlib.transforms.ScaledTranslation(25/72, -15/72, fig.dpi_scale_trans)
    end
    
    
    ax_i.text(0.0, 1.0, string("(", label, ")"), fontsize=30, transform=ax_i.transAxes + trans, 
        verticalalignment="top")
    
end

plt.subplots_adjust(hspace=0.45, wspace=0.3)

plt.savefig("Holding_period.png", dpi=300, bbox_inches="tight")

- We observe that in-sample optimized portfolios are suboptimal in the testing period (expected).

In [ ]:
## Sharpe ratio

Sharpe_Ratio = SharpeRatio(results[:, 1], results[:, 2])

mSharpe_Ratio = mSharpeRatio(results[:, 1], results[:, 2], results[:, 3], results[:, 4], 0.1)

Sharpe_Ratio_p_OOS_1 = SharpeRatio(results_p_opt_1[:, 1], results_p_opt_1[:, 2])

mSharpe_Ratio_p_OOS_1 = mSharpeRatio(results_p_opt_1[:, 1], results_p_opt_1[:, 2], 
    results_p_opt_1[:, 3], results_p_opt_1[:, 4], 0.1)


λ_vals = 0.0 : 0.05 : 1.0

mean_SR_λ = zeros((length(λ_vals), 4))
mean_mSR_λ = zeros((length(λ_vals), 4))
mean_SR_λ_test = zeros((length(λ_vals), 4))
mean_mSR_λ_test = zeros((length(λ_vals), 4))

k = 0

for i in λ_vals

    k += 1
    
    mean_SR_λ[k, 1] = mean(Sharpe_Ratio[λs_test_1[:, 1] .== i])
    mean_SR_λ[k, 2] = mean(Sharpe_Ratio[λs_test_1[:, 2] .== i])
    mean_SR_λ[k, 3] = mean(Sharpe_Ratio[λs_test_1[:, 3] .== i])
    mean_SR_λ[k, 4] = mean(Sharpe_Ratio[λs_test_1[:, 4] .== i])
    
    mean_mSR_λ[k, 1] = mean(mSharpe_Ratio[λs_test_1[:, 1] .== i])
    mean_mSR_λ[k, 2] = mean(mSharpe_Ratio[λs_test_1[:, 2] .== i])
    mean_mSR_λ[k, 3] = mean(mSharpe_Ratio[λs_test_1[:, 3] .== i])
    mean_mSR_λ[k, 4] = mean(mSharpe_Ratio[λs_test_1[:, 4] .== i])
    
    mean_SR_λ_test[k, 1] = mean(Sharpe_Ratio_p_OOS_1[λs_test_1[:, 1] .== i])
    mean_SR_λ_test[k, 2] = mean(Sharpe_Ratio_p_OOS_1[λs_test_1[:, 2] .== i])
    mean_SR_λ_test[k, 3] = mean(Sharpe_Ratio_p_OOS_1[λs_test_1[:, 3] .== i])
    mean_SR_λ_test[k, 4] = mean(Sharpe_Ratio_p_OOS_1[λs_test_1[:, 4] .== i])
    
    mean_mSR_λ_test[k, 1] = mean(mSharpe_Ratio_p_OOS_1[λs_test_1[:, 1] .== i])
    mean_mSR_λ_test[k, 2] = mean(mSharpe_Ratio_p_OOS_1[λs_test_1[:, 2] .== i])
    mean_mSR_λ_test[k, 3] = mean(mSharpe_Ratio_p_OOS_1[λs_test_1[:, 3] .== i])
    mean_mSR_λ_test[k, 4] = mean(mSharpe_Ratio_p_OOS_1[λs_test_1[:, 4] .== i])
    
end

##

idxs = 1 : length(Sharpe_Ratio_p_OOS_1)

SR_MV = Sharpe_Ratio_p_OOS_1[(λs_test_1[:, 3] .== 0.0) .& (λs_test_1[:, 4] .== 0.0)]
idxs_MV = idxs[(λs_test_1[:, 3] .== 0.0) .& (λs_test_1[:, 4] .== 0.0)]

SR_λ4_05 = Sharpe_Ratio_p_OOS_1[(λs_test_1[:, 4] .== 0.9)]
idxs_λ4_05 = idxs[(λs_test_1[:, 4] .== 0.9)]

SR_λ3_02 = Sharpe_Ratio_p_OOS_1[(λs_test_1[:, 3] .== 0.1)]
idxs_λ3_02 = idxs[(λs_test_1[:, 3] .== 0.1)];

max_SR = maximum(results_OOS_MV_opt_1[:, 1] ./ results_OOS_MV_opt_1[:, 2]);

idxs_SR_ordered = sortperm(Sharpe_Ratio, rev=true)
idxs_mSR_ordered = sortperm(mSharpe_Ratio, rev=true)
idxs_SR_ordered_test = sortperm(Sharpe_Ratio_p_OOS_1, rev=true)
idxs_mSR_ordered_test = sortperm(mSharpe_Ratio_p_OOS_1, rev=true)

λs_max_SR = λs[idxs_SR_ordered, :][1:10, :]
Max_10_SR = round.(Sharpe_Ratio[idxs_SR_ordered][1:10], digits=3)

λs_max_mSR = λs[idxs_mSR_ordered, :][1:10, :]
Max_10_mSR = round.(mSharpe_Ratio[idxs_mSR_ordered][1:10], digits=3)

λs_max_SR_test = λs_test_1[idxs_SR_ordered_test, :][1:10, :]
Max_10_SR_test = round.(Sharpe_Ratio_p_OOS_1[idxs_SR_ordered_test][1:10], digits=3)

λs_max_mSR_test = λs_test_1[idxs_mSR_ordered_test, :][1:10, :]
Max_10_mSR_test = round.(mSharpe_Ratio_p_OOS_1[idxs_mSR_ordered_test][1:10], digits=3)

println("Max. Top 10 SR: ", maximum(Max_10_SR))
println("Min. Top 10 SR: ", minimum(Max_10_SR))
println("λs:", λs_max_SR[1:3, :])

println("\nMax. Top 10 mSR: ", maximum(Max_10_mSR))
println("Min. Top 10 mSR: ", minimum(Max_10_mSR))
println("λs:", λs_max_mSR[1:3, :])

println("\nTEST\n")

println("Max. Top 10 SR: ", maximum(Max_10_SR_test))
println("Min. Top 10 SR: ", minimum(Max_10_SR_test))
println("λs:", λs_max_SR_test[1:3, :])

println("\nMax. Top 10 mSR: ", maximum(Max_10_mSR_test))
println("Min. Top 10 mSR: ", minimum(Max_10_mSR_test))
println("λs:", λs_max_mSR_test[1:3, :])

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
CD""", figsize=(8*2,6*2))

ax["A"].plot(λ_vals, mean_SR_λ[:, 1], lw=3, label=L"$\lambda_1$")
ax["A"].plot(λ_vals, mean_SR_λ[:, 2], lw=3, label=L"$\lambda_2$")
ax["A"].plot(λ_vals, mean_SR_λ[:, 3], lw=3, label=L"$\lambda_3$")
ax["A"].plot(λ_vals, mean_SR_λ[:, 4], lw=3, label=L"$\lambda_4$")

ax["A"].set_xlabel(L"$\lambda$ value")
ax["A"].set_ylabel("Sharpe Ratio")

ax["A"].legend(loc="upper center", ncol=4, bbox_to_anchor=(1, 1.22))

colors = matplotlib.cm.get_cmap("jet_r", 12)

xx = [0, 0.33, 0.67,1.0]

for i in 10 : -1 : 1
   
    ax["B"].plot(xx, λs_max_SR[i, :], color=colors(i+1), marker="o", ms=10, label=L"$SR=%$(Max_10_SR[i])$")
    
end

ax["B"].set_xticks(xx, [L"$\lambda_1$", L"$\lambda_2$", L"$\lambda_3$", L"$\lambda_4$"], fontsize=24)

ax["B"].set_ylabel(L"$\lambda$ value")

cax = fig.add_axes([0.92, 0.54, 0.025, 0.35])

cmap = matplotlib.cm.get_cmap("jet", 12)
norm = matplotlib.colors.Normalize(vmin=minimum(Max_10_SR), vmax=maximum(Max_10_SR))

cb1 = matplotlib.colorbar.ColorbarBase(cax, cmap=cmap, norm=norm)

cb1.ax.set_title("SR")

ax["C"].plot(λ_vals, mean_mSR_λ[:, 1], lw=3, label=L"$\lambda_1$")
ax["C"].plot(λ_vals, mean_mSR_λ[:, 2], lw=3, label=L"$\lambda_2$")
ax["C"].plot(λ_vals, mean_mSR_λ[:, 3], lw=3, label=L"$\lambda_3$")
ax["C"].plot(λ_vals, mean_mSR_λ[:, 4], lw=3, label=L"$\lambda_4$")

ax["C"].set_xlabel(L"$\lambda$ value")
ax["C"].set_ylabel("Modified Sharpe Ratio")

colors = matplotlib.cm.get_cmap("jet_r", 12)

xx = [0, 0.33, 0.67,1.0]

for i in 10 : -1 : 1
   
    ax["D"].plot(xx, λs_max_mSR[i, :], color=colors(i+1), marker="o", ms=10, label=L"$SR=%$(Max_10_mSR[i])$")
    
end

ax["D"].set_xticks(xx, [L"$\lambda_1$", L"$\lambda_2$", L"$\lambda_3$", L"$\lambda_4$"], fontsize=24)

ax["D"].set_ylabel(L"$\lambda$ value")

cax2 = fig.add_axes([0.92, 0.09, 0.025, 0.35])

cmap = matplotlib.cm.get_cmap("jet", 12)
norm = matplotlib.colors.Normalize(vmin=minimum(Max_10_mSR), vmax=maximum(Max_10_mSR))

cb2 = matplotlib.colorbar.ColorbarBase(cax2, cmap=cmap, norm=norm)

cb2.ax.set_title("mSR")

#Labelling
for item in ax
    
    label, ax_i = item
    
    # label physical distance in and down:
    trans = matplotlib.transforms.ScaledTranslation(25/72, -15/72, fig.dpi_scale_trans)
    ax_i.text(0.0, 1.0, string("(", label, ")"), fontsize=30, transform=ax_i.transAxes + trans, 
        verticalalignment="top")
    
end

plt.subplots_adjust(hspace=0.3, wspace=0.3)

plt.savefig("In_sample_optimal_portfolios.png", dpi=300, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
CD""", figsize=(8*2,6*2))

ax["A"].plot(λ_vals, mean_SR_λ_test[:, 1], lw=3, label=L"$\lambda_1$")
ax["A"].plot(λ_vals, mean_SR_λ_test[:, 2], lw=3, label=L"$\lambda_2$")
ax["A"].plot(λ_vals, mean_SR_λ_test[:, 3], lw=3, label=L"$\lambda_3$")
ax["A"].plot(λ_vals, mean_SR_λ_test[:, 4], lw=3, label=L"$\lambda_4$")

ax["A"].set_xlabel(L"$\lambda$ value")
ax["A"].set_ylabel("Sharpe Ratio")

ax["A"].legend(loc="upper center", ncol=4, bbox_to_anchor=(1, 1.22))

colors = matplotlib.cm.get_cmap("jet_r", 12)

xx = [0, 0.33, 0.67,1.0]

for i in 10 : -1 : 1
   
    ax["B"].plot(xx, λs_max_SR_test[i, :], color=colors(i+1), marker="o", ms=10, label=L"$SR=%$(Max_10_SR[i])$")
    
end

ax["B"].set_xticks(xx, [L"$\lambda_1$", L"$\lambda_2$", L"$\lambda_3$", L"$\lambda_4$"], fontsize=24)

ax["B"].set_ylabel(L"$\lambda$ value")

cax = fig.add_axes([0.92, 0.54, 0.025, 0.35])

cmap = matplotlib.cm.get_cmap("jet", 12)
norm = matplotlib.colors.Normalize(vmin=minimum(Max_10_SR), vmax=maximum(Max_10_SR))

cb1 = matplotlib.colorbar.ColorbarBase(cax, cmap=cmap, norm=norm)

cb1.ax.set_title("SR")

ax["C"].plot(λ_vals, mean_mSR_λ_test[:, 1], lw=3, label=L"$\lambda_1$")
ax["C"].plot(λ_vals, mean_mSR_λ_test[:, 2], lw=3, label=L"$\lambda_2$")
ax["C"].plot(λ_vals, mean_mSR_λ_test[:, 3], lw=3, label=L"$\lambda_3$")
ax["C"].plot(λ_vals, mean_mSR_λ_test[:, 4], lw=3, label=L"$\lambda_4$")

ax["C"].set_xlabel(L"$\lambda$ value")
ax["C"].set_ylabel("Modified Sharpe Ratio")

colors = matplotlib.cm.get_cmap("jet_r", 12)

xx = [0, 0.33, 0.67,1.0]

for i in 10 : -1 : 1
   
    ax["D"].plot(xx, λs_max_mSR_test[i, :], color=colors(i+1), marker="o", ms=10, label=L"$SR=%$(Max_10_mSR[i])$")
    
end

ax["D"].set_xticks(xx, [L"$\lambda_1$", L"$\lambda_2$", L"$\lambda_3$", L"$\lambda_4$"], fontsize=24)

ax["D"].set_ylabel(L"$\lambda$ value")

cax2 = fig.add_axes([0.92, 0.09, 0.025, 0.35])

cmap = matplotlib.cm.get_cmap("jet", 12)
norm = matplotlib.colors.Normalize(vmin=minimum(Max_10_mSR), vmax=maximum(Max_10_mSR))

cb2 = matplotlib.colorbar.ColorbarBase(cax2, cmap=cmap, norm=norm)

cb2.ax.set_title("mSR")

#Labelling
for item in ax
    
    label, ax_i = item
    
    # label physical distance in and down:
    trans = matplotlib.transforms.ScaledTranslation(25/72, -15/72, fig.dpi_scale_trans)
    ax_i.text(0.0, 1.0, string("(", label, ")"), fontsize=30, transform=ax_i.transAxes + trans, 
        verticalalignment="top")
    
end

plt.subplots_adjust(hspace=0.3, wspace=0.3)

plt.savefig("Out_of_sample_optimal_portfolios.png", dpi=300, bbox_inches="tight")

- In the sampling period, MV-optimized portfolios are the ones yielding higher Sharpe Ratios and modified Sharpe ratios. 
- However, when looking at the testing period we observe that accounting for skewness and kurtosis during the sampling period (MVSK-optimized portfolios) yields more robust portfolios. This is, MVSK-optimized portfolios are the ones outperforming in the testing period looking both at their Sharpe Ratios and modified Sharpe Ratios. 

# Dynamic Portfolio Optimization 

### Download data 

In [ ]:
start_date = Date("2000-01-01")
end_date = Date("2022-01-01")

freq = "1M"

df_market = yf.download("^GSPC", start_date, end_date, progress=false).resample(freq).mean()

returns_market = df_market["Adj Close"].pct_change().dropna().values;

### Preliminar analisis 

In [ ]:
#Load optimized portfolios
filenames = readdir("HD_efficient_frontier_2")

results = zeros((length(filenames), 4))
λs = zeros((length(filenames), 4))
weights = zeros((length(filenames), 15))

i = 1

for filename in filenames
   
    data, header = readdlm(string("HD_efficient_frontier_2/", filename), header=true)
   
    results[i, :] = data[1:4]
    λs[i, :] = data[5:8]
    weights[i, :] = data[9:end]
    
    i += 1
    
end

results_MV = sort(results[(λs[:, 3] .== 0.0) .& (λs[:, 4] .== 0.0), :], dims=1)
results_MK = sort(results[(λs[:, 2] .== 0.0) .& (λs[:, 3] .== 0.0), :], dims=1)
results_SK = sort(results[(λs[:, 1] .== 0.0) .& (λs[:, 2] .== 0.0), :], dims=1);

SR = SharpeRatio(results[:, 1], results[:, 2])
mSR = mSharpeRatio(results[:, 1], results[:, 2], results[:, 3], results[:, 4], 0.1)

best_λs_SR = λs[sortperm(SR, rev=true), :]
best_λs_mSR = λs[sortperm(mSR, rev=true), :]

println("Best λ SR: ", best_λs_SR[1, :])
println("Best λ mSR: ", best_λs_mSR[1, :])

### Effect of the lookback period 

We use the following $\lambda$ values so that risk is equally weighted for both models and study the effect of the lookback period length in the dynamic optimization problem

- λ_MV = 0.8
- λ_MVSK = [0.2, 0.65, 0.1, 0.05]

In [ ]:
##Load Results
start_date = Date("2000-01-01")

fig, ax = plt.subplot_mosaic("""AB
    CD
    EF""", figsize=(8*2, 6*3))

legend_elements = [L2D([0], [0], lw=3, color="C0", label="MV-D"),
                   L2D([0], [0], lw=3, color="C1", label="MV-S"),
                   L2D([0], [0], lw=3, color="C2", label="MVSK-D"),
                   L2D([0], [0], lw=3, color="C3", label="MVSK-S"),
                   L2D([0], [0], lw=3, color="C4", label="EW")]

       
returns = readdlm("Dynamic_Optimization_Results/Returns_$(start_date).txt")
volatility = readdlm("Dynamic_Optimization_Results/Volatility_$(start_date).txt")
skew = readdlm("Dynamic_Optimization_Results/Skewness_$(start_date).txt")
kurt = readdlm("Dynamic_Optimization_Results/Kurtosis_$(start_date).txt")

max_MDD = readdlm("Dynamic_Optimization_Results/Max_MDD_$(start_date).txt")
mean_MDD = readdlm("Dynamic_Optimization_Results/Mean_MDD_$(start_date).txt")
mean_RoMaD = readdlm("Dynamic_Optimization_Results/Mean_RoMaD_$(start_date).txt")
 
ax["A"].plot(1:10, returns.*100, lw=3, marker="o", ms=12)
ax["A"].set_ylabel("Returns [%]")
ax["A"].set_xlabel("Lookback period [Years]")

ax["A"].legend(handles=legend_elements, loc="upper left", ncol=5, bbox_to_anchor=(0.2, 1.35))

ax["B"].plot(1:10, volatility.*100, lw=3, marker="o", ms=12)
ax["B"].set_ylabel("Volatility [%]")
ax["B"].set_xlabel("Lookback period [Years]")

ax["C"].plot(1:10, skew, lw=3, marker="o", ms=12)
ax["C"].set_ylabel("Skewness")
ax["C"].set_xlabel("Lookback period [Years]")

ax["D"].plot(1:10, kurt, lw=3, marker="o", ms=12)
ax["D"].set_ylabel("Kurtosis")
ax["D"].set_xlabel("Lookback period [Years]")

ax["E"].plot(1:10, SharpeRatio(returns, volatility), lw=3, marker="o", ms=12)
ax["E"].set_ylabel("Sharpe Ratio")
ax["E"].set_xlabel("Lookback period [Years]")

ax["F"].plot(1:10, mSharpeRatio(returns, volatility, skew, kurt, 0.1), lw=3, marker="o", ms=12)
ax["F"].set_ylabel("Modified Sharpe Ratio")
ax["F"].set_xlabel("Lookback period [Years]")

#Labelling
for item in ax
    
    label, ax_i = item
    
    # label physical distance in and down:
    trans = matplotlib.transforms.ScaledTranslation(4.5, -15/72, fig.dpi_scale_trans)
    ax_i.text(0.0, 1.0, string("(", label, ")"), fontsize=30, transform=ax_i.transAxes + trans, 
        verticalalignment="top")
    
end

plt.subplots_adjust(hspace=0.4, wspace=0.3)

plt.savefig("Lookback_period_effect.png", dpi=300, bbox_inches="tight")

In general we can conclude that longer lookback periods do not improve portfolio performance. In particular, we observe that 1 and 2 year lookback periods are best for MV-D and MVSK-D optimization procedures. However, we observe that mid-term lookback periods (-5 years) can help static models to increase performance

### Example 

We use the following $\lambda$ values so that risk is equally weighted for both models and study the effect of the lookback period length in the dynamic optimization problem

- λ_MV = 0.95
- λ_MVSK = [0.9, 0.05, 0.05, 0.0]

In [ ]:
realized_returns = readdlm("Dynamic_Optimization_Results/realized_returns_2000-01-01_2022-01-01.txt")
portfolio_value = readdlm("Dynamic_Optimization_Results/portfolio_value_2000-01-01_2022-01-01.txt")
weights = readdlm("Dynamic_Optimization_Results/weights_2000-01-01_2022-01-01.txt");

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
    CD""", figsize=(8*2, 6*2))

legend_elements = [L2D([0], [0], lw=3, color="C0", label="MV-D"),
                   L2D([0], [0], lw=3, color="C1", label="MV-S"),
                   L2D([0], [0], lw=3, color="C2", label="MVSK-D"),
                   L2D([0], [0], lw=3, color="C3", label="MVSK-S"),
                   L2D([0], [0], lw=3, color="C4", label="EW"),
                   L2D([0], [0], lw=3, color="k", label="S&P 500")]

dates = start_date  + Year(1) : Month(24) : end_date

market_value = df_market["Adj Close"].values / df_market["Adj Close"].values[1]

ax["A"].plot(1:size(portfolio_value)[1], portfolio_value[:, 1], color="C0", lw=3, label="MV-D");
ax["A"].plot(1:size(portfolio_value)[1], portfolio_value[:, 2], color="C1", lw=3, label="MV-S")
ax["A"].plot(1:size(portfolio_value)[1], portfolio_value[:, 3], color="C2", lw=3, label="MVSK-D")
ax["A"].plot(1:size(portfolio_value)[1], portfolio_value[:, 4], color="C3", lw=3, label="MVSK-S")
ax["A"].plot(1:size(portfolio_value)[1], portfolio_value[:, 5], color="C4", lw=3, label="EW")

ax["A"].plot(1:size(portfolio_value)[1], market_value, color="k", lw=3)

ax["A"].set_ylabel("Portfolio value")
ax["A"].set_xticks(1:24:size(realized_returns)[1], 2000:2:2021, rotation=45)

ax["A"].set_yscale("log")

ax["A"].legend(handles=legend_elements, loc="upper left", ncol=6, bbox_to_anchor=(-0.05, 1.25))

ax["B"].boxplot([realized_returns[:, 1], realized_returns[:, 2], realized_returns[:, 3], realized_returns[:, 4],
        realized_returns[:, 5], returns_market])

ax["B"].set_xticks([1, 2, 3, 4, 5, 6], ["MV-D", "MV-S", "MVSK-D", "MVSK-S", "EW", "SP500"])
ax["B"].set_ylabel("Returns")

max_year_drawdown = zeros((Int(round(Dates.value(end_date - start_date) / (30*12))), 5))
RoMaD = zeros((Int(round(Dates.value(end_date - start_date) / (30*12))), 5))

max_year_drawdown_market = zeros((Int(round(Dates.value(end_date - start_date) / (30*12)))))
RoMaD_market = zeros((Int(round(Dates.value(end_date - start_date) / (30*12)))))

for i in 1 : 22
    
    portfolio_value_i = portfolio_value[12*(i-1) + 1 : 12*i, :]
    market_value_i = market_value[12*(i-1) + 1 : 12*i]
    
    DD, MDD = maximum_drawdown(portfolio_value_i)
 
    DD_market, MDD_market = maximum_drawdown(reshape(market_value_i, (length(market_value_i), 1)))
    
    mean_return = mean(realized_returns[12*(i-1) + 1 : 12*i, :], dims=1) * 100
    
    mean_return_market = mean(returns_market[12*(i-1) + 1 : 12*i]) * 100
        
    max_year_drawdown[i, :] = MDD
    RoMaD[i, :] = - mean_return ./ MDD
            
    max_year_drawdown_market[i, :] = MDD_market
    RoMaD_market[i, :] = - mean_return_market ./ MDD_market
    
end

RoMaD[argmax(filter(!isinf, RoMaD))] = 6
RoMaD[RoMaD .== Inf] .= 6

ax["C"].plot(max_year_drawdown, marker="o", lw=3)
ax["C"].plot(max_year_drawdown_market, marker="o", lw=3, color="k")
        
ax["C"].set_xticks(0:2:21, 2000:2:2021, rotation=45)
ax["C"].set_ylabel("Monthly MDD [%]")

ax["D"].plot(RoMaD, marker="o", lw=3)
ax["D"].plot(RoMaD_market, marker="o", lw=3, color="k")

ax["D"].set_ylabel("RoMaD")

ax["D"].set_xticks(0:2:21, 2000:2:2021, rotation=45)

#Labelling
for item in ax
    
    label, ax_i = item
    
    # label physical distance in and down:
    trans = matplotlib.transforms.ScaledTranslation(10/72, -15/72, fig.dpi_scale_trans)
    ax_i.text(0.0, 1.0, string("(", label, ")"), fontsize=30, transform=ax_i.transAxes + trans, 
        verticalalignment="top")
    
end

plt.subplots_adjust(hspace=0.25, wspace=0.3)

mSRs_10 = []
mSRs_5 = []
mSRs_1 = []

for i in 1:5
    
    ret = mean(realized_returns[:, i])
    risk = sqrt(var(realized_returns[:, i]))
    skew = skewness(realized_returns[:, i])
    kurt = kurtosis(realized_returns[:, i])
    
    append!(mSRs_10, mSharpeRatio(ret, risk, skew, kurt, 0.1))
    append!(mSRs_5, mSharpeRatio(ret, risk, skew, kurt, 0.05))
    append!(mSRs_1, mSharpeRatio(ret, risk, skew, kurt, 0.01))
    
end

ret = mean(returns_market)
risk = sqrt(var(returns_market))
skew = skewness(returns_market)
kurt = kurtosis(returns_market)

append!(mSRs_10, mSharpeRatio(ret, risk, skew, kurt, 0.10))
append!(mSRs_5, mSharpeRatio(ret, risk, skew, kurt, 0.05))
append!(mSRs_1, mSharpeRatio(ret, risk, skew, kurt, 0.01))


#=
println("Mean returns: ", ["$(i)%" for i in mean(realized_returns, dims=1).*100])
println("Mean volatility: ", ["$(i)%" for i in sqrt.(var(realized_returns, dims=1)) .* 100])
println("Sharpe ratio: ",  mean(realized_returns, dims=1) ./ sqrt.(var(realized_returns, dims=1)))
println("Modified Sharpe Ratio: ", mSRs)
println("Max MDD", ["$(i)" for i in minimum(max_year_drawdown, dims=1)])
println("Mean MDD", ["$(i)" for i in mean(max_year_drawdown, dims=1)])
println("Mean RoMaD", ["$(i)" for i in mean(RoMaD, dims=1)])

println("\nMarket:", [mean(returns_market)*100, sqrt(var(returns_market))*100, 
        mean(returns_market) / sqrt(var(returns_market))])
=#

returns = vcat(vec(mean(realized_returns, dims=1)), mean(returns_market))
risk = vcat(vec(sqrt.(var(realized_returns, dims=1))), sqrt(var(returns_market)))
skew = vcat(vec([skewness(realized_returns[:,i]) for i in 1 : 5]), skewness(returns_market))
kurt = vcat(vec([kurtosis(realized_returns[:,i]) for i in 1 : 5]), kurtosis(returns_market))

SR = vcat(vec(mean(realized_returns, dims=1) ./ sqrt.(var(realized_returns, dims=1))),
    mean(returns_market) / sqrt(var(returns_market)))

results_dict = OrderedDict("Return [%]" => returns*100, "Risk [%]" => risk*100, "Skewness" => skew, "Kurtosis" => kurt, 
    "SharpeRatio" => SR, "Modified SR (10%)" => mSRs_10, "Modified SR (5%)" => mSRs_5, "Modified SR (1%)" => mSRs_1)

df_results = pd.DataFrame(results_dict, index=["MV-D", "MV-S", "MVSK-D", "MVSK-S", "EW", "SP500"])

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
    CD""", figsize=(8*4, 6*4))

moments_returns = zeros((4, 5))

for i in 1:5
    
    moments_returns[1, i] = mean(realized_returns[:, i])
    moments_returns[2, i] = sqrt(var(realized_returns[:, i]))
    moments_returns[3, i] = skewness(realized_returns[:, i])
    moments_returns[4, i] = kurtosis(realized_returns[:, i])
    
end

ax["A"].bar(1:5, moments_returns[1, :])

ax["A"].set_xticks(1:5, ["MV-D", "MV-S", "MVSK-D", "MVSK-S", "EW"], fontsize=30)

ax["A"].tick_params(axis="y", which="major", labelsize=24)


ax["B"].bar(1:5, moments_returns[2, :])

ax["B"].set_xticks(1:5, ["MV-D", "MV-S", "MVSK-D", "MVSK-S", "EW"], fontsize=30)

ax["B"].tick_params(axis="y", which="major", labelsize=24)


ax["C"].bar(1:5, moments_returns[3, :])

ax["C"].set_xticks(1:5, ["MV-D", "MV-S", "MVSK-D", "MVSK-S", "EW"], fontsize=30)

ax["C"].tick_params(axis="y", which="major", labelsize=24)


ax["D"].bar(1:5, moments_returns[4, :])

ax["D"].set_xticks(1:5, ["MV-D", "MV-S", "MVSK-D", "MVSK-S", "EW"], fontsize=30)

ax["D"].tick_params(axis="y", which="major", labelsize=24)

#Labelling
for item in ax
    
    label, ax_i = item
    
    # label physical distance in and down:
    trans = matplotlib.transforms.ScaledTranslation(10/72, -15/72, fig.dpi_scale_trans)
    ax_i.text(0.0, 1.0, string("(", label, ")"), fontsize=30, transform=ax_i.transAxes + trans, 
        verticalalignment="top")
    
end

### Example 2 

We use the following $\lambda$ values so that all moments are equally weighted for both models and study the effect of the lookback period length in the dynamic optimization problem

- λ_MV = 0.5
- λ_MVSK = [0.25, 0.25, 0.25, 0.25]

In [ ]:
realized_returns = readdlm("Dynamic_Optimization_Results/realized_returns_2000-01-01_2022-01-01_2.txt")
portfolio_value = readdlm("Dynamic_Optimization_Results/portfolio_value_2000-01-01_2022-01-01_2.txt")
weights = readdlm("Dynamic_Optimization_Results/weights_2000-01-01_2022-01-01_2.txt");

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
    CD""", figsize=(8*2, 6*2))

legend_elements = [L2D([0], [0], lw=3, color="C0", label="MV-D"),
                   L2D([0], [0], lw=3, color="C1", label="MV-S"),
                   L2D([0], [0], lw=3, color="C2", label="MVSK-D"),
                   L2D([0], [0], lw=3, color="C3", label="MVSK-S"),
                   L2D([0], [0], lw=3, color="C4", label="EW"),
                   L2D([0], [0], lw=3, color="k", label="S&P 500")]

dates = start_date  + Year(1) : Month(24) : end_date

market_value = df_market["Adj Close"].values / df_market["Adj Close"].values[1]

ax["A"].plot(1:size(portfolio_value)[1], portfolio_value[:, 1], color="C0", lw=3, label="MV-D");
ax["A"].plot(1:size(portfolio_value)[1], portfolio_value[:, 2], color="C1", lw=3, label="MV-S")
ax["A"].plot(1:size(portfolio_value)[1], portfolio_value[:, 3], color="C2", lw=3, label="MVSK-D")
ax["A"].plot(1:size(portfolio_value)[1], portfolio_value[:, 4], color="C3", lw=3, label="MVSK-S")
ax["A"].plot(1:size(portfolio_value)[1], portfolio_value[:, 5], color="C4", lw=3, label="EW")

ax["A"].plot(1:size(portfolio_value)[1], market_value, color="k", lw=3)

ax["A"].set_ylabel("Portfolio value")
ax["A"].set_xticks(1:24:size(realized_returns)[1], 2000:2:2021, rotation=45)

ax["A"].set_yscale("log")

ax["A"].legend(handles=legend_elements, loc="upper left", ncol=6, bbox_to_anchor=(-0.05, 1.25))

ax["B"].boxplot([realized_returns[:, 1], realized_returns[:, 2], realized_returns[:, 3], realized_returns[:, 4],
        realized_returns[:, 5], returns_market])

ax["B"].set_xticks([1, 2, 3, 4, 5, 6], ["MV-D", "MV-S", "MVSK-D", "MVSK-S", "EW", "SP500"])
ax["B"].set_ylabel("Returns")

max_year_drawdown = zeros((Int(round(Dates.value(end_date - start_date) / (30*12))), 5))
RoMaD = zeros((Int(round(Dates.value(end_date - start_date) / (30*12))), 5))

max_year_drawdown_market = zeros((Int(round(Dates.value(end_date - start_date) / (30*12)))))
RoMaD_market = zeros((Int(round(Dates.value(end_date - start_date) / (30*12)))))

for i in 1 : 22
    
    portfolio_value_i = portfolio_value[12*(i-1) + 1 : 12*i, :]
    market_value_i = market_value[12*(i-1) + 1 : 12*i]
    
    DD, MDD = maximum_drawdown(portfolio_value_i)
 
    DD_market, MDD_market = maximum_drawdown(reshape(market_value_i, (length(market_value_i), 1)))
    
    mean_return = mean(realized_returns[12*(i-1) + 1 : 12*i, :], dims=1) * 100
    
    mean_return_market = mean(returns_market[12*(i-1) + 1 : 12*i]) * 100
        
    max_year_drawdown[i, :] = MDD
    RoMaD[i, :] = - mean_return ./ MDD
            
    max_year_drawdown_market[i, :] = MDD_market
    RoMaD_market[i, :] = - mean_return_market ./ MDD_market
    
end

RoMaD[argmax(filter(!isinf, RoMaD))] = 6
RoMaD[RoMaD .== Inf] .= 6

ax["C"].plot(max_year_drawdown, marker="o", lw=3)
ax["C"].plot(max_year_drawdown_market, marker="o", lw=3, color="k")
        
ax["C"].set_xticks(0:2:21, 2000:2:2021, rotation=45)
ax["C"].set_ylabel("Monthly MDD [%]")

ax["D"].plot(RoMaD, marker="o", lw=3)
ax["D"].plot(RoMaD_market, marker="o", lw=3, color="k")

ax["D"].set_ylabel("RoMaD")

ax["D"].set_xticks(0:2:21, 2000:2:2021, rotation=45)

#Labelling
for item in ax
    
    label, ax_i = item
    
    # label physical distance in and down:
    trans = matplotlib.transforms.ScaledTranslation(10/72, -15/72, fig.dpi_scale_trans)
    ax_i.text(0.0, 1.0, string("(", label, ")"), fontsize=30, transform=ax_i.transAxes + trans, 
        verticalalignment="top")
    
end

plt.subplots_adjust(hspace=0.25, wspace=0.3)

mSRs_10 = []
mSRs_5 = []
mSRs_1 = []

for i in 1:5
    
    ret = mean(realized_returns[:, i])
    risk = sqrt(var(realized_returns[:, i]))
    skew = skewness(realized_returns[:, i])
    kurt = kurtosis(realized_returns[:, i])
    
    append!(mSRs_10, mSharpeRatio(ret, risk, skew, kurt, 0.1))
    append!(mSRs_5, mSharpeRatio(ret, risk, skew, kurt, 0.05))
    append!(mSRs_1, mSharpeRatio(ret, risk, skew, kurt, 0.01))
    
end

ret = mean(returns_market)
risk = sqrt(var(returns_market))
skew = skewness(returns_market)
kurt = kurtosis(returns_market)

append!(mSRs_10, mSharpeRatio(ret, risk, skew, kurt, 0.10))
append!(mSRs_5, mSharpeRatio(ret, risk, skew, kurt, 0.05))
append!(mSRs_1, mSharpeRatio(ret, risk, skew, kurt, 0.01))


#=
println("Mean returns: ", ["$(i)%" for i in mean(realized_returns, dims=1).*100])
println("Mean volatility: ", ["$(i)%" for i in sqrt.(var(realized_returns, dims=1)) .* 100])
println("Sharpe ratio: ",  mean(realized_returns, dims=1) ./ sqrt.(var(realized_returns, dims=1)))
println("Modified Sharpe Ratio: ", mSRs)
println("Max MDD", ["$(i)" for i in minimum(max_year_drawdown, dims=1)])
println("Mean MDD", ["$(i)" for i in mean(max_year_drawdown, dims=1)])
println("Mean RoMaD", ["$(i)" for i in mean(RoMaD, dims=1)])

println("\nMarket:", [mean(returns_market)*100, sqrt(var(returns_market))*100, 
        mean(returns_market) / sqrt(var(returns_market))])
=#

returns = vcat(vec(mean(realized_returns, dims=1)), mean(returns_market))
risk = vcat(vec(sqrt.(var(realized_returns, dims=1))), sqrt(var(returns_market)))
skew = vcat(vec([skewness(realized_returns[:,i]) for i in 1 : 5]), skewness(returns_market))
kurt = vcat(vec([kurtosis(realized_returns[:,i]) for i in 1 : 5]), kurtosis(returns_market))

SR = vcat(vec(mean(realized_returns, dims=1) ./ sqrt.(var(realized_returns, dims=1))),
    mean(returns_market) / sqrt(var(returns_market)))

results_dict = OrderedDict("Return [%]" => returns*100, "Risk [%]" => risk*100, "Skewness" => skew, "Kurtosis" => kurt, 
    "SharpeRatio" => SR, "Modified SR (10%)" => mSRs_10, "Modified SR (5%)" => mSRs_5, "Modified SR (1%)" => mSRs_1)

df_results = pd.DataFrame(results_dict, index=["MV-D", "MV-S", "MVSK-D", "MVSK-S", "EW", "SP500"])

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
    CD""", figsize=(8*4, 6*4))

moments_returns = zeros((4, 5))

for i in 1:5
    
    moments_returns[1, i] = mean(realized_returns[:, i])
    moments_returns[2, i] = sqrt(var(realized_returns[:, i]))
    moments_returns[3, i] = skewness(realized_returns[:, i])
    moments_returns[4, i] = kurtosis(realized_returns[:, i])
    
end

ax["A"].bar(1:5, moments_returns[1, :])

ax["A"].set_xticks(1:5, ["MV-D", "MV-S", "MVSK-D", "MVSK-S", "EW"], fontsize=30)

ax["A"].tick_params(axis="y", which="major", labelsize=24)


ax["B"].bar(1:5, moments_returns[2, :])

ax["B"].set_xticks(1:5, ["MV-D", "MV-S", "MVSK-D", "MVSK-S", "EW"], fontsize=30)

ax["B"].tick_params(axis="y", which="major", labelsize=24)


ax["C"].bar(1:5, moments_returns[3, :])

ax["C"].set_xticks(1:5, ["MV-D", "MV-S", "MVSK-D", "MVSK-S", "EW"], fontsize=30)

ax["C"].tick_params(axis="y", which="major", labelsize=24)


ax["D"].bar(1:5, moments_returns[4, :])

ax["D"].set_xticks(1:5, ["MV-D", "MV-S", "MVSK-D", "MVSK-S", "EW"], fontsize=30)

ax["D"].tick_params(axis="y", which="major", labelsize=24)

#Labelling
for item in ax
    
    label, ax_i = item
    
    # label physical distance in and down:
    trans = matplotlib.transforms.ScaledTranslation(10/72, -15/72, fig.dpi_scale_trans)
    ax_i.text(0.0, 1.0, string("(", label, ")"), fontsize=30, transform=ax_i.transAxes + trans, 
        verticalalignment="top")
    
end

# Yield Curve 

In [ ]:
function clean_df(df)
    
    df["Date"] = pd.to_datetime(df["Date"])
    
    df = df.set_index("Date")
    
    df = df.drop(["20 YR", "30 YR", "Extrapolation Factor",
       "8 WEEKS BANK DISCOUNT", "COUPON EQUIVALENT", "52 WEEKS BANK DISCOUNT",
       "COUPON EQUIVALENT.1","2 Mo"], axis=1)
    
    return df

end

function get_YC_data(years)

    year = years[1]

    url = "https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value=$(year)"

    YC_df = clean_df(pd.read_html(url)[1])

    for year in years[2:end]

        url = "https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value=$(year)"

        if year == years[1]

            YC_df = clean_df(pd.read_html(url)[1])

        else

            YC_df = pd.concat([YC_df, clean_df(pd.read_html(url)[1])])

        end

    end
        
    return YC_df
    
end

In [ ]:
#2011-2022 period
years_train = 2013 : 1 : 2016

YC_df_train = get_YC_data(years_train)

#2017-2020 period
years_test = 2017 : 1 : 2019

YC_df_test = get_YC_data(years_test)

#Long 1990-2022 period
years = 1990: 1 : 2022

YC_df = get_YC_data(years)

#YC_df_avg = YC_df.resample("1Y").mean();

YC_df_train_avg = YC_df_train.resample("1Y").mean()
YC_df_test_avg = YC_df_test.resample("1Y").mean();

In [ ]:
colors = matplotlib.cm.get_cmap("viridis", length(YC_df.columns))

colnames = [item for item in YC_df.columns]

legend_elements = []

for i in 1 : length(YC_df.columns)
   
    append!(legend_elements, [L2D([0], [0], color=colors(i), lw=5, label=colnames[i])])
    
end

append!(legend_elements, [L2D([0], [0], color="b", ls=":", lw=2, label="10Yr-2Yr spread")])

fig, ax = plt.subplot_mosaic("""AA
    BC""", figsize=((8*2, 6*2.2)))

YC_df.plot(ax=ax["A"], cmap="viridis")

#COVID crisis
circle1 = Ellipse((YC_df.index.values[7550], 1.4), 300, 3.5, fill=false, color="red", lw=3, ls="-", zorder=2)

ax["A"].text(YC_df.index.values[6700], 6, "COVID-19 crisis")
ax["A"].annotate("", (YC_df.index.values[7550], 3.1), (YC_df.index.values[7550], 5.8), 
    arrowprops=Dict("arrowstyle"=>"->", "color"=>"k", "linewidth"=>2),)
ax["A"].add_patch(circle1)

#2008 crisis (Great Recession)
circle2 = Ellipse((YC_df.index.values[4550], 2.9), 500, 6, fill=false, color="red", lw=3, zorder=2)

ax["A"].text(YC_df.index.values[3200], 9, "Great Recession")
ax["A"].annotate("", (YC_df.index.values[4400], 6), (YC_df.index.values[4100], 8.8), 
    arrowprops=Dict("arrowstyle"=>"->", "color"=>"k", "linewidth"=>2),)
ax["A"].add_patch(circle2)

#2000 crisis (Dot-com bubble)
circle3 = Ellipse((YC_df.index.values[2500], 4), 1200, 8.5, fill=false, color="red", lw=3, zorder=2)

ax["A"].text(YC_df.index.values[1200], 9, "Dot-com bubble")
ax["A"].annotate("", (YC_df.index.values[2400], 7), (YC_df.index.values[2000], 8.8), 
    arrowprops=Dict("arrowstyle"=>"->", "color"=>"k", "linewidth"=>2),)
ax["A"].add_patch(circle3)

ax["A"].set_xlabel("")
ax["A"].set_ylabel("Yield [%]")

ax2 = ax["A"].twinx()

ax2.plot(YC_df["10 Yr"] - YC_df["2 Yr"], color="b", ls=":", alpha=1, label="10Yr-2Yr spread")

ax2.set_ylabel("Spread", color="b")

ax["A"].legend(handles=legend_elements, ncol=6, loc="upper center", bbox_to_anchor=(0.5, 1.32))

for i in 1 : length(YC_df_train_avg)

    YC_df_train_avg.iloc[i].plot(ax=ax["B"], label="$(years_train[i])", lw=3, marker="o")
        
    #ax["B"].legend(ncol=3, loc="upper center", bbox_to_anchor=(0.5, 1.35))
    ax["B"].legend(ncol=1, loc="upper left")
    
end

for i in 1 : length(YC_df_test_avg)

    YC_df_test_avg.iloc[i].plot(ax=ax["C"], label="$(years_test[i])", lw=3, marker="o")
    
    ax["C"].legend(ncol=1, loc="lower right")#, bbox_to_anchor=(0.5, 1.35))
    
end

ax["B"].set_ylabel("Yield [%]")
ax["C"].set_ylabel("Yield [%]")

#Labelling
for item in ax
    
    label, ax_i = item

    if label == "A"
    
        trans = matplotlib.transforms.ScaledTranslation(11.5, -15/72, fig.dpi_scale_trans)
    
    elseif label == "B"
        
        trans = matplotlib.transforms.ScaledTranslation(2.8, -15/72, fig.dpi_scale_trans)
        
    else
        
        trans = matplotlib.transforms.ScaledTranslation(10/72, -15/72, fig.dpi_scale_trans)
        
    end
        
    ax_i.text(0.0, 1.0, string("(", label, ")"), fontsize=30, transform=ax_i.transAxes + trans, 
        verticalalignment="top")
    
end

plt.subplots_adjust(hspace=0.25, wspace=0.25)

plt.savefig("Economic_situation.png", dpi=300, bbox_inches="tight")

In [ ]:
spread = YC_df["10 Yr"] - YC_df["2 Yr"]

# MV-Dynamic with all US stocks

In [ ]:
function dynamic_MV(returns, assets, lookback_period, λ; w_upper=1.0)

    periods = size(returns)[1] - lookback_period
    
    realized_returns = zeros(periods-1)

    portfolio_value = ones(periods) 
    
    weights = zeros((periods-1, length(assets)))
    
    returns_i = returns[1 : lookback_period, :]

    portfolio_i = create_portfolio(returns_i, assets; higher_moments=false);

    ret, risk, w_opt = MV(portfolio_i, λ; method="DCP", w_upper=w_upper)

    for i in 1 : periods-1

        #Check performance        
        returns_delta = returns[lookback_period + i - 1 : lookback_period + i - 1, :]

        returns_p = (returns_delta * w_opt)[1]

        realized_returns[i] = returns_p

        weights[i, :] = w_opt

        #Recompute optimal strategy 
        returns_i = returns[1+i - 1 : lookback_period + i - 1, :]

        portfolio_i = create_portfolio(returns_i, assets; higher_moments=false);

        ret, risk, w_opt = MV(portfolio_i, λ; method="DCP", w_upper=w_upper)

        portfolio_value[i+1] = portfolio_value[i, 1] * (1+returns_p)    

    end

    return realized_returns, portfolio_value, weights
    
end

### All data optimization 

In [ ]:
df = pd.read_csv("US_stocks_2010_2022.csv", index_col=0)

assets = [item for item in df.columns];

## Perform calculations in cluster!
#lookback_period = 12 * 1
#λ = 0.8

#realized_returns, portfolio_value, weights = @time dynamic_MV(returns, assets, lookback_period, λ; w_upper=1.0)

realized_returns_λ_02 = readdlm("Dynamic_Optimization_Results/realized_returns_alldata_MV_λ_0.2.txt")
realized_returns_λ_05 = readdlm("Dynamic_Optimization_Results/realized_returns_alldata_MV_λ_0.5.txt")
realized_returns_λ_08 = readdlm("Dynamic_Optimization_Results/realized_returns_alldata_MV_λ_0.8.txt")
realized_returns_λ_09 = readdlm("Dynamic_Optimization_Results/realized_returns_alldata_MV_λ_0.9.txt")
realized_returns_λ_095 = readdlm("Dynamic_Optimization_Results/realized_returns_alldata_MV_λ_0.95.txt")
realized_returns_λ_099 = readdlm("Dynamic_Optimization_Results/realized_returns_alldata_MV_λ_0.99.txt")

portfolio_value_λ_02 = readdlm("Dynamic_Optimization_Results/portfolio_value_alldata_MV_λ_0.2.txt")
portfolio_value_λ_05 = readdlm("Dynamic_Optimization_Results/portfolio_value_alldata_MV_λ_0.5.txt")
portfolio_value_λ_08 = readdlm("Dynamic_Optimization_Results/portfolio_value_alldata_MV_λ_0.8.txt")
portfolio_value_λ_09 = readdlm("Dynamic_Optimization_Results/portfolio_value_alldata_MV_λ_0.9.txt")
portfolio_value_λ_095 = readdlm("Dynamic_Optimization_Results/portfolio_value_alldata_MV_λ_0.95.txt")
portfolio_value_λ_099 = readdlm("Dynamic_Optimization_Results/portfolio_value_alldata_MV_λ_0.99.txt")

weights_λ_02 = readdlm("Dynamic_Optimization_Results/weights_alldata_MV_λ_0.2.txt")
weights_λ_05 = readdlm("Dynamic_Optimization_Results/weights_alldata_MV_λ_0.5.txt")
weights_λ_08 = readdlm("Dynamic_Optimization_Results/weights_alldata_MV_λ_0.8.txt")
weights_λ_09 = readdlm("Dynamic_Optimization_Results/weights_alldata_MV_λ_0.9.txt")
weights_λ_095 = readdlm("Dynamic_Optimization_Results/weights_alldata_MV_λ_0.95.txt")
weights_λ_099 = readdlm("Dynamic_Optimization_Results/weights_alldata_MV_λ_0.99.txt");

In [ ]:
portfolio_value = zeros((length(portfolio_value_λ_02), 6))
realized_returns = zeros((length(realized_returns_λ_02), 6))

portfolio_value[:, 1] = portfolio_value_λ_02
portfolio_value[:, 2] = portfolio_value_λ_05
portfolio_value[:, 3] = portfolio_value_λ_08
portfolio_value[:, 4] = portfolio_value_λ_09
portfolio_value[:, 5] = portfolio_value_λ_095
portfolio_value[:, 6] = portfolio_value_λ_099

realized_returns[:, 1] = realized_returns_λ_02
realized_returns[:, 2] = realized_returns_λ_05
realized_returns[:, 3] = realized_returns_λ_08
realized_returns[:, 4] = realized_returns_λ_09
realized_returns[:, 5] = realized_returns_λ_095
realized_returns[:, 6] = realized_returns_λ_099;

In [ ]:
start_date = Date("2011-01-01")
end_date = Date("2022-01-01")

freq = "1M"

df_market = yf.download("^GSPC", start_date, end_date, progress=false).resample(freq).mean()
returns_market = df_market["Adj Close"].pct_change().dropna().values;

plt.figure(figsize=(8*2, 6*2))

dates = start_date  + Year(1) : Month(24) : end_date

market_value = df_market["Adj Close"].values / df_market["Adj Close"].values[1]

plt.subplot(2, 2, 1)

plt.plot(portfolio_value, lw=3);

plt.plot(market_value, color="k", lw=3)

plt.ylabel("Portfolio value")
plt.xticks(1:24:size(realized_returns_λ_02)[1], 2011:2:2022, rotation=45)

plt.yscale("log")

plt.subplot(2,2,2)

plt.boxplot([realized_returns_λ_02[:,1], realized_returns_λ_05[:,1], realized_returns_λ_08[:,1],
        realized_returns_λ_09[:,1], realized_returns_λ_095[:,1], realized_returns_λ_099[:,1], returns_market])

plt.xticks([1, 2, 3, 4, 5, 6, 7], [L"$\lambda=0.2$", L"$\lambda=0.5$", L"$\lambda=0.8$", L"$\lambda=0.9$",
        L"$\lambda=0.95$", L"$\lambda=0.99$", "SP500"], rotation=45)
plt.ylabel("Returns")

plt.subplot(2, 2, 3)

max_year_drawdown = zeros((Int(round(Dates.value(end_date - start_date) / (30*12))), 6))
RoMaD = zeros((Int(round(Dates.value(end_date - start_date) / (30*12))), 6))

max_year_drawdown_market = zeros((Int(round(Dates.value(end_date - start_date) / (30*12)))))
RoMaD_market = zeros((Int(round(Dates.value(end_date - start_date) / (30*12)))))

for i in 1 : 10
    
    portfolio_value_i = portfolio_value[12*(i-1) + 1 : 12*i, :]
    market_value_i = market_value[12*(i-1) + 1 : 12*i]
    
    DD, MDD = maximum_drawdown(portfolio_value_i)
 
    DD_market, MDD_market = maximum_drawdown(reshape(market_value_i, (length(market_value_i), 1)))
    
    mean_return = mean(realized_returns[12*(i-1) + 1 : 12*i], dims=1) * 100
    
    mean_return_market = mean(returns_market[12*(i-1) + 1 : 12*i]) * 100
        
    max_year_drawdown[i, :] = MDD
    RoMaD[i, :] = - mean_return ./ MDD
            
    max_year_drawdown_market[i, :] = MDD_market
    RoMaD_market[i, :] = - mean_return_market ./ MDD_market
    
end

RoMaD[argmax(filter(!isinf, RoMaD))] = 6
RoMaD[RoMaD .== Inf] .= 6

plt.plot(max_year_drawdown, marker="o", lw=3)
plt.plot(max_year_drawdown_market, marker="o", lw=3, color="k")
        
plt.xticks(0:1:11, 2011:1:2022, rotation=45)
plt.ylabel("Monthly MDD [%]")

plt.subplot(2, 2, 4)

plt.plot(RoMaD, marker="o", lw=3)
plt.plot(RoMaD_market, marker="o", lw=3, color="k")

plt.ylabel("RoMaD")

plt.xticks(0:1:11, 2011:1:2022, rotation=45)

plt.subplots_adjust(hspace=0.25, wspace=0.3)

mSRs_10 = []
mSRs_5 = []
mSRs_1 = []

for i in 1:6
    
    ret = mean(realized_returns[:, i])
    risk = sqrt(var(realized_returns[:, i]))
    skew = skewness(realized_returns[:, i])
    kurt = kurtosis(realized_returns[:, i])
    
    append!(mSRs_10, mSharpeRatio(ret, risk, skew, kurt, 0.1))
    append!(mSRs_5, mSharpeRatio(ret, risk, skew, kurt, 0.05))
    append!(mSRs_1, mSharpeRatio(ret, risk, skew, kurt, 0.01))
    
end

ret = mean(returns_market)
risk = sqrt(var(returns_market))
skew = skewness(returns_market)
kurt = kurtosis(returns_market)

append!(mSRs_10, mSharpeRatio(ret, risk, skew, kurt, 0.10))
append!(mSRs_5, mSharpeRatio(ret, risk, skew, kurt, 0.05))
append!(mSRs_1, mSharpeRatio(ret, risk, skew, kurt, 0.01))

returns = vcat(vec(mean(realized_returns, dims=1)), mean(returns_market))
risk = vcat(vec(sqrt.(var(realized_returns, dims=1))), sqrt(var(returns_market)))
skew = vcat(vec([skewness(realized_returns[:,i]) for i in 1 : 6]), skewness(returns_market))
kurt = vcat(vec([kurtosis(realized_returns[:,i]) for i in 1 : 6]), kurtosis(returns_market))

SR = vcat(vec(mean(realized_returns, dims=1) ./ sqrt.(var(realized_returns, dims=1))),
    mean(returns_market) / sqrt(var(returns_market)))

results_dict = OrderedDict("Return [%]" => returns*100, "Risk [%]" => risk*100, "Skewness" => skew, "Kurtosis" => kurt, 
    "SharpeRatio" => SR, "Modified SR (10%)" => mSRs_10, "Modified SR (5%)" => mSRs_5, "Modified SR (1%)" => mSRs_1)

df_results = pd.DataFrame(results_dict, index=["λ=0.2", "λ=0.5", "λ=0.8", "λ=0.9", "λ=0.95", "λ=0.99", "S&P500"])

In [ ]:
anim = @animate for idx in 1 : size(realized_returns_λ_02)[1]

    sorted_weights_λ_02 = sort(weights_λ_02[idx, :], rev=true)
    sorted_weights_λ_05 = sort(weights_λ_05[idx, :], rev=true)
    sorted_weights_λ_08 = sort(weights_λ_08[idx, :], rev=true)
    sorted_weights_λ_09 = sort(weights_λ_09[idx, :], rev=true)

    sorted_assets_λ_02 = assets[sortperm(weights_λ_02[idx, :], rev=true)]
    sorted_assets_λ_05 = assets[sortperm(weights_λ_05[idx, :], rev=true)]
    sorted_assets_λ_08 = assets[sortperm(weights_λ_08[idx, :], rev=true)]
    sorted_assets_λ_09 = assets[sortperm(weights_λ_09[idx, :], rev=true)]

    p1 = Plots.bar(1:10, sorted_weights_λ_02[1:10], xticks=(1:10, sorted_assets_λ_02[1:10]), framestyle=:box, 
        ylim=(0,1.05), ylabel="Weights", labelfontsize=20, tickfontsize=14, legend=:false, 
            title=L"\lambda=0.2", titlefontsize=24, xrotation=30)

    p2 = Plots.bar(1:10, sorted_weights_λ_05[1:10], xticks=(1:10, sorted_assets_λ_05[1:10]), framestyle=:box, 
        ylim=(0,1.05), ylabel="Weights", labelfontsize=20, tickfontsize=14, legend=:false, 
            title=L"\lambda=0.2", titlefontsize=24, xrotation=30)

    p3 = Plots.bar(1:10, sorted_weights_λ_08[1:10], xticks=(1:10, sorted_assets_λ_08[1:10]), framestyle=:box, 
        ylim=(0,1.05), ylabel="Weights", labelfontsize=20, tickfontsize=14, legend=:false, 
            title=L"\lambda=0.2", titlefontsize=24, xrotation=30)

    p4 = Plots.bar(1:10, sorted_weights_λ_09[1:10], xticks=(1:10, sorted_assets_λ_09[1:10]), framestyle=:box, 
        ylim=(0,1.05), ylabel="Weights", labelfontsize=20, tickfontsize=14, legend=:false, 
            title=L"\lambda=0.2", titlefontsize=24)

    Plots.plot(p1, p2, p3, p4, layout=4, size=(1600, 1200), plot_title=Date("2010-01-01") + Month(idx-1), 
        plot_titlefontsize=30, margin=5mm)
    
end

gif(anim, "weight_evolution_alldata.gif", fps=2)

### Top 10 assets optimization for different w_upper

In [ ]:
lookback_period = 12 * 1
λ = 0.95

df = pd.read_csv("US_stocks_2010_2022.csv", index_col=0)

assets = [item for item in df.columns]
        
returns = df.pct_change().dropna().values

#if higher_moments == true, large memory is needed ofr allocation
portfolio = create_portfolio(returns[1:12*8, :], assets; higher_moments=false)

#Top 10 assets Fortune 500 year 2010
top_10_assets = ["WMT", "XOM", "CVX",  "GE", "BAC", "COP", "T", "F", "JPM", "HP"]

idxs_top_10_assets = []

idxs = 1 : size(returns)[2]

for i in 1 : 10
    
    append!(idxs_top_10_assets, idxs[assets .== top_10_assets[i]])
    
end

returns_top_10 = returns[:, idxs_top_10_assets]

realized_returns_top_10_w = zeros((size(returns)[1] - lookback_period - 1, 4))
portfolio_value_top_10_w = zeros((size(returns)[1] - lookback_period, 4))
weights_top_10_w = zeros((size(returns)[1] - lookback_period - 1, 10, 4))

i = 0

@time for w_upper in [1.0, 0.5, 0.25, 0.1]
   
    i += 1
    
    realized_returns_top_10, portfolio_value_top_10, weights_top_10 = dynamic_MV(returns_top_10, top_10_assets, lookback_period,
    λ; w_upper=w_upper)
    
    realized_returns_top_10_w[:, i] = realized_returns_top_10
    portfolio_value_top_10_w[:, i] = portfolio_value_top_10
    weights_top_10_w[:, :, i] = weights_top_10
    
end

In [ ]:
start_date = Date("2011-02-01")
end_date = Date("2022-01-01")

freq = "1M"

df_market = yf.download("^GSPC", start_date, end_date, progress=false).resample(freq).mean()
returns_market = df_market["Adj Close"].pct_change().dropna().values;

plt.figure(figsize=(8*2, 6*2))

legend_elements = [L2D([0], [0], lw=3, color="C0", label=L"$w_{max}=1.0$"),
                   L2D([0], [0], lw=3, color="C1", label=L"$w_{max}=0.5$"),
                   L2D([0], [0], lw=3, color="C2", label=L"$w_{max}=0.25$"),
                   L2D([0], [0], lw=3, color="C3", label=L"$w_{max}=0.1$"),
                   L2D([0], [0], lw=3, color="k", label="S&P 500")]

dates = start_date  + Year(1) : Month(24) : end_date

market_value = df_market["Adj Close"].values / df_market["Adj Close"].values[1]

plt.subplot(2, 2, 1)

plt.plot(1:size(portfolio_value_top_10_w)[1], portfolio_value_top_10_w[:, 1], lw=3, label=L"$w_{max}=1.0$");
plt.plot(1:size(portfolio_value_top_10_w)[1], portfolio_value_top_10_w[:, 2], lw=3, label=L"$w_{max}=0.5$")
plt.plot(1:size(portfolio_value_top_10_w)[1], portfolio_value_top_10_w[:, 3], lw=3, label=L"$w_{max}=0.25$")
plt.plot(1:size(portfolio_value_top_10_w)[1], portfolio_value_top_10_w[:, 4], lw=3, label=L"$w_{max}=0.1$")

plt.plot(1:size(portfolio_value_top_10_w)[1], market_value, color="k", lw=3)

plt.ylabel("Portfolio value")
plt.xticks(1:24:size(realized_returns_top_10_w)[1], 2011:2:2022, rotation=45)

#plt.yscale("log")

plt.legend(handles=legend_elements, loc="upper left", ncol=6, bbox_to_anchor=(-0.05, 1.25))

plt.subplot(2,2,2)

plt.boxplot([realized_returns_top_10_w[:, 1], realized_returns_top_10_w[:, 2], realized_returns_top_10_w[:, 3],
        realized_returns_top_10_w[:, 4], returns_market])

plt.xticks([1, 2, 3, 4, 5], 
    [L"$w_{max}=1.0$", L"$w_{max}=0.5$", L"$w_{max}=0.25$", L"$w_{max}=0.1$", "SP500"], rotation=45)
plt.ylabel("Returns")

plt.subplot(2, 2, 3)

max_year_drawdown = zeros((Int(round(Dates.value(end_date - start_date) / (30*12))), 4))
RoMaD = zeros((Int(round(Dates.value(end_date - start_date) / (30*12))), 4))

max_year_drawdown_market = zeros((Int(round(Dates.value(end_date - start_date) / (30*12)))))
RoMaD_market = zeros((Int(round(Dates.value(end_date - start_date) / (30*12)))))

for i in 1 : 10
    
    portfolio_value_top_10_w_i = portfolio_value_top_10_w[12*(i-1) + 1 : 12*i, :]
    market_value_i = market_value[12*(i-1) + 1 : 12*i]
    
    DD, MDD = maximum_drawdown(portfolio_value_top_10_w_i)
 
    DD_market, MDD_market = maximum_drawdown(reshape(market_value_i, (length(market_value_i), 1)))
    
    mean_return = mean(realized_returns_top_10_w[12*(i-1) + 1 : 12*i, :], dims=1) * 100
    
    mean_return_market = mean(returns_market[12*(i-1) + 1 : 12*i]) * 100
        
    max_year_drawdown[i, :] = MDD
    RoMaD[i, :] = - mean_return ./ MDD
            
    max_year_drawdown_market[i, :] = MDD_market
    RoMaD_market[i, :] = - mean_return_market ./ MDD_market
    
end

RoMaD[argmax(filter(!isinf, RoMaD))] = 6
RoMaD[RoMaD .== Inf] .= 6

plt.plot(max_year_drawdown, marker="o", lw=3)
plt.plot(max_year_drawdown_market, marker="o", lw=3, color="k")
        
plt.xticks(0:2:11, 2011:2:2022, rotation=45)
plt.ylabel("Monthly MDD [%]")

plt.subplot(2, 2, 4)

plt.plot(RoMaD, marker="o", lw=3)
plt.plot(RoMaD_market, marker="o", lw=3, color="k")

plt.ylabel("RoMaD")

plt.xticks(0:2:11, 2011:2:2022, rotation=45)

plt.subplots_adjust(hspace=0.3, wspace=0.3)

mSRs_10 = []
mSRs_5 = []
mSRs_1 = []

for i in 1:4
    
    ret = mean(realized_returns_top_10_w[:, i])
    risk = sqrt(var(realized_returns_top_10_w[:, i]))
    skew = skewness(realized_returns_top_10_w[:, i])
    kurt = kurtosis(realized_returns_top_10_w[:, i])
    
    append!(mSRs_10, mSharpeRatio(ret, risk, skew, kurt, 0.1))
    append!(mSRs_5, mSharpeRatio(ret, risk, skew, kurt, 0.05))
    append!(mSRs_1, mSharpeRatio(ret, risk, skew, kurt, 0.01))
    
end

ret = mean(returns_market)
risk = sqrt(var(returns_market))
skew = skewness(returns_market)
kurt = kurtosis(returns_market)

append!(mSRs_10, mSharpeRatio(ret, risk, skew, kurt, 0.10))
append!(mSRs_5, mSharpeRatio(ret, risk, skew, kurt, 0.05))
append!(mSRs_1, mSharpeRatio(ret, risk, skew, kurt, 0.01))

returns = vcat(vec(mean(realized_returns_top_10_w, dims=1)), mean(returns_market))
risk = vcat(vec(sqrt.(var(realized_returns_top_10_w, dims=1))), sqrt(var(returns_market)))
skew = vcat(vec([skewness(realized_returns_top_10_w[:,i]) for i in 1 : 4]), skewness(returns_market))
kurt = vcat(vec([kurtosis(realized_returns_top_10_w[:,i]) for i in 1 : 4]), kurtosis(returns_market))

SR = vcat(vec(mean(realized_returns_top_10_w, dims=1) ./ sqrt.(var(realized_returns_top_10_w, dims=1))),
    mean(returns_market) / sqrt(var(returns_market)))

results_dict = OrderedDict("Return [%]" => returns*100, "Risk [%]" => risk*100, "Skewness" => skew, "Kurtosis" => kurt, 
    "SharpeRatio" => SR, "Modified SR (10%)" => mSRs_10, "Modified SR (5%)" => mSRs_5, "Modified SR (1%)" => mSRs_1)

df_results = pd.DataFrame(results_dict, index=["ω_max=1.0", "ω_max=0.5", "ω_max=0.25", "ω_max=0.1", "S&P500"])

### Weight changes 

In [ ]:
anim = @animate for i in 1 : size(weights_top_10_w)[1]
    
    p1 = Plots.bar(1:10, weights_top_10_w[i, :, 1], xticks=(1:10, top_10_assets), framestyle=:box, 
    ylim=(0,1.05), ylabel="Weights", labelfontsize=20, tickfontsize=14, legend=:false, 
        title=L"w_{max}=1.0", titlefontsize=24)
    
    p2 = Plots.bar(1:10, weights_top_10_w[i, :, 2], xticks=(1:10, top_10_assets), framestyle=:box, 
    ylim=(0,1.05), ylabel="Weights", labelfontsize=20, tickfontsize=14, legend=:false, 
        title=L"w_{max}=0.5", titlefontsize=24)
    
    p3 = Plots.bar(1:10, weights_top_10_w[i, :, 3], xticks=(1:10, top_10_assets), framestyle=:box,  
    ylim=(0,1.05), ylabel="Weights", labelfontsize=20, tickfontsize=14, legend=:false, 
        title=L"w_{max}=0.25", titlefontsize=24)
    
    p4 = Plots.bar(1:10, weights_top_10_w[i, :, 4], xticks=(1:10, top_10_assets), framestyle=:box, 
    ylim=(0,1.05), ylabel="Weights", labelfontsize=20, tickfontsize=14, legend=:false, 
        title=L"w_{max}=0.1", titlefontsize=24)
    
    Plots.plot(p1, p2, p3, p4, layout=4, size=(1600, 1200), plot_title=Date("2010-01-01") + Month(i-1), 
    plot_titlefontsize=30, margin=5mm)
            
end

gif(anim, "weight_evolution_w.gif", fps=2)

# Auxiliar things 

In [ ]:
returns = df["Adj Close"].pct_change().dropna().values

returns_test_1 = df_test_1["Adj Close"].pct_change().dropna().values
returns_test_2 = df_test_2["Adj Close"].pct_change().dropna().values
returns_test_3 = df_test_3["Adj Close"].pct_change().dropna().values

assets = string.(zeros((1, 10)))

i = 0

for item in df["Adj Close"].columns
   
    i += 1
    
    assets[i] = item
    
end

f = open("returns.txt", "w")

f1 = open("returns_test_1.txt", "w")
f2 = open("returns_test_2.txt", "w")
f3 = open("returns_test_3.txt", "w")

#f2 = open("log_returns.txt", "w")
#f4 = open("log_returns_test.txt", "w")

writedlm(f, assets)
writedlm(f, returns)

writedlm(f1, assets)
writedlm(f1, returns_test_1)

writedlm(f2, assets)
writedlm(f2, returns_test_2)

writedlm(f3, assets)
writedlm(f3, returns_test_3)

close(f)

close(f1)
close(f2)
close(f3)

In [ ]:
start_date = Date("1990-01-01")
end_date = Date("2000-01-01")

freq = "1M"

#Top-10 in Fortune 500 list from year 2000 
##General Motors is top 1 but only available since 2010
assets = ["WMT", "XOM", "F", "GE", "IBM", "C", "T", "MO", "BA", "BAC", "HPQ", "KR", "AIG", "HD", "PG"] 

df = yf.download(assets, start_date, end_date, progress=false).resample(freq).mean()

returns = df["Adj Close"].pct_change().dropna().values

assets = string.(zeros((1, 15)))

i = 0

for item in df["Adj Close"].columns
   
    i += 1
    
    assets[i] = item
    
end

f = open("returns_2.txt", "w")

writedlm(f, assets)
writedlm(f, returns)

close(f)

In [ ]:
λs = []

δ = 0.02

for i in 0.0 : δ : 1.0
    
    for j in 0.0 : δ : 1.0
        
        for k in 0.0 : δ : 1.0
            
            for l in 0.0 : δ : 1.0
                
                if i+j+k+l == 1.0
                   
                    append!(λs, [[i,j,k,l]])
                    
                end
                
            end
            
        end
        
    end
    
end

λs

In [ ]:
λs = hcat(0.0 : 0.1 : 1.0, 1.0 : -0.1 : 0.0, zeros(11), zeros(11))

In [ ]:
RESULTS = zeros((size(λs)[1], 4))

for i in 1 : size(λs)[1]

    println("λ=", λs[i, :])
    
    res, weights = @time MVSK(portfolio_test_1, λs[i, :])
    
    RESULTS[i, :] = res
    
end

In [ ]:
plt.plot(RetRisk_test_1[:, 2], RetRisk_test_1[:, 1], ls="-", lw=5, color="r")

plt.scatter(RESULTS[:, 2], RESULTS[:, 1], s=200)

plt.scatter(results_OOS_MV_opt_1[:,2], results_OOS_MV_opt_1[:, 1], color="g", s=200) 

In [ ]:
using Distributions

μ_for_mean(m, σ) = log(m) - σ^2/2

In [ ]:
σs = [10^i for i in -2 : 0.01 : 0.5]

μ = 0.1

N = 10^6

means = zeros(size(σs))
vars  = zeros(size(σs))
skews = zeros(size(σs))

weighted_means = zeros(size(σs))
modified_risk = zeros(size(σs))

i = 0

@time for σ in σs
    
    i += 1

    d = LogNormal(μ_for_mean(μ, σ), σ)

    returns = rand(d, N)
    
    prob, val = np.histogram(returns, bins=1000, density=false)
    
    means[i] = mean(returns)
    vars[i] = var(returns)
    skews[i] = skewness(returns)
    
    weighted_means[i] = sum(prob ./ N .* val[2:end])
    modified_risk[i] = sum(prob[val[2:end] .< weighted_means[i]] ./ N)
    
end

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
    CD""", figsize=(8*2, 6*2))

ax["A"].plot(σs, means, lw=3)
ax["A"].plot(σs, weighted_means, lw=3)

ax["A"].set_yscale("log")

ax["B"].plot(σs, vars, lw=3)
ax["B"].set_yscale("log")

ax["C"].plot(σs, skews, lw=3)

ax["C"].set_yscale("log")

ax["D"].plot(σs, modified_risk, lw=3)


In [ ]:
means ./ vars

In [ ]:
skews ./ vars

In [ ]:
N = 10^6

returns = 0.1 .* randn(N) .+ 0.05

prob, val = np.histogram(returns, bins=1000, density=false)

println(mean(returns))
println(var(returns))
println(sum(prob/N .* val[2:end]))

# TESTS 

In [ ]:
L = 10^5
M = 10

returns = rand(L, M)

P = create_portfolio(returns, ["A" for i in 1 : M]);

In [ ]:
w = rand(M)

w = w ./ sum(w)

p = returns * w;

In [ ]:
println(mean(p))
println(return_portfolio(P, w))

println("\n")

println(sqrt(var(p)))
println(risk_portfolio(P, w))

println("\n")

println(moment(p, 3))
println(skewness_portfolio(P, w))

println("\n")

println(moment(p, 4))
println(kurtosis_portfolio(P, w))

In [ ]:
println(mean(p))
println(return_portfolio(P, w))

println("\n")

println(sqrt(var(p)))
println(risk_portfolio(P, w))

println("\n")

println(skewness(p))
println(standarized_skewness_portfolio(P, w))

println("\n")

println(kurtosis(p) + 3)
println(standarized_kurtosis_portfolio(P, w))

println("\n")

println(kurtosis(p))
println(standarized_excess_kurtosis_portfolio(P, w))